In [2]:
import os
import struct
import argparse
from typing import List, Dict, Any
import ctypes
import mmap
import tkinter as tk
from tkinter import filedialog, ttk
import threading

# Low-level filesystem access
class RawDisk:
    def __init__(self, device_path):
        self.device_path = device_path
        self.fd = os.open(device_path, os.O_RDONLY)
        self.mm = mmap.mmap(self.fd, 0, prot=mmap.PROT_READ)

    def read(self, offset, size):
        return self.mm[offset:offset+size]

    def close(self):
        self.mm.close()
        os.close(self.fd)

# File signature detection
FILE_SIGNATURES = {
    b'\xFF\xD8\xFF': 'jpg',
    b'\x89\x50\x4E\x47': 'png',
    b'\x25\x50\x44\x46': 'pdf',
    b'\x50\x4B\x03\x04': 'zip',
    # Add more signatures as needed
}

def detect_file_type(data):
    for signature, file_type in FILE_SIGNATURES.items():
        if data.startswith(signature):
            return file_type
    return 'unknown'

class FileSystemRecoveryTool:
    def __init__(self, device_path: str, filesystem_type: str):
        self.device_path = device_path
        self.filesystem_type = filesystem_type
        self.device = None

    def open_device(self):
        try:
            self.device = RawDisk(self.device_path)
        except IOError as e:
            raise IOError(f"Error opening device: {e}")

    def close_device(self):
        if self.device:
            self.device.close()

    def read_superblock(self) -> Dict[str, Any]:
        if self.filesystem_type == 'xfs':
            return self.read_xfs_superblock()
        elif self.filesystem_type == 'btrfs':
            return self.read_btrfs_superblock()
        else:
            raise ValueError("Unsupported filesystem type")

    def read_xfs_superblock(self) -> Dict[str, Any]:
        superblock_data = self.device.read(0, 512)
        magic = struct.unpack_from('I', superblock_data, 0)[0]
        if magic != 0x58465342:  # 'XFSB'
            raise ValueError("Invalid XFS superblock")
        # Extract other relevant information...
        return {'magic': magic}

    def read_btrfs_superblock(self) -> Dict[str, Any]:
        superblock_data = self.device.read(65536, 4096)
        magic = struct.unpack_from('8s', superblock_data, 64)[0]
        if magic != b'_BHRfS_M':
            raise ValueError("Invalid Btrfs superblock")
        # Extract other relevant information...
        return {'magic': magic}

    def scan_for_deleted_files(self) -> List[Dict[str, Any]]:
        deleted_files = []
        chunk_size = 4096
        offset = 0
        while True:
            chunk = self.device.read(offset, chunk_size)
            if not chunk:
                break
            file_type = detect_file_type(chunk)
            if file_type != 'unknown':
                deleted_files.append({
                    'offset': offset,
                    'type': file_type,
                    'size': self.carve_file(offset, file_type)
                })
            offset += chunk_size
        return deleted_files

    def carve_file(self, start_offset: int, file_type: str) -> int:
        # Simple file carving - read until end of file signature or max size
        max_size = 10 * 1024 * 1024  # 10 MB max file size for this example
        chunk_size = 4096
        file_data = b''
        offset = start_offset
        while len(file_data) < max_size:
            chunk = self.device.read(offset, chunk_size)
            if not chunk:
                break
            file_data += chunk
            offset += chunk_size
            # Check for end of file signature (simplified)
            if file_type == 'jpg' and b'\xFF\xD9' in chunk:
                break
            # Add more end-of-file checks for other file types
        return len(file_data)

    def recover_file(self, file_info: Dict[str, Any], output_dir: str) -> bool:
        try:
            data = self.device.read(file_info['offset'], file_info['size'])
            output_path = os.path.join(output_dir, f"recovered_{file_info['offset']}.{file_info['type']}")
            with open(output_path, 'wb') as f:
                f.write(data)
            return True
        except IOError as e:
            print(f"Error recovering file at offset {file_info['offset']}: {e}")
            return False

    def extract_metadata(self, file_info: Dict[str, Any]) -> Dict[str, Any]:
        # This would involve parsing filesystem-specific structures
        # For this example, we'll just return some basic info
        return {
            'offset': file_info['offset'],
            'size': file_info['size'],
            'type': file_info['type'],
        }

# GUI Implementation
class RecoveryGUI:
    def __init__(self, master):
        self.master = master
        master.title("Filesystem Recovery Tool")

        self.device_label = tk.Label(master, text="Device Path:")
        self.device_label.grid(row=0, column=0, sticky="e")
        self.device_entry = tk.Entry(master, width=50)
        self.device_entry.grid(row=0, column=1)
        self.device_button = tk.Button(master, text="Browse", command=self.browse_device)
        self.device_button.grid(row=0, column=2)

        self.fs_type_label = tk.Label(master, text="Filesystem Type:")
        self.fs_type_label.grid(row=1, column=0, sticky="e")
        self.fs_type_var = tk.StringVar(value="xfs")
        self.fs_type_menu = tk.OptionMenu(master, self.fs_type_var, "xfs", "btrfs")
        self.fs_type_menu.grid(row=1, column=1, sticky="w")

        self.output_label = tk.Label(master, text="Output Directory:")
        self.output_label.grid(row=2, column=0, sticky="e")
        self.output_entry = tk.Entry(master, width=50)
        self.output_entry.grid(row=2, column=1)
        self.output_button = tk.Button(master, text="Browse", command=self.browse_output)
        self.output_button.grid(row=2, column=2)

        self.start_button = tk.Button(master, text="Start Recovery", command=self.start_recovery)
        self.start_button.grid(row=3, column=1)

        self.progress = ttk.Progressbar(master, length=300, mode='indeterminate')
        self.progress.grid(row=4, column=0, columnspan=3, pady=10)

        self.status_label = tk.Label(master, text="")
        self.status_label.grid(row=5, column=0, columnspan=3)

    def browse_device(self):
        filename = filedialog.askopenfilename()
        self.device_entry.delete(0, tk.END)
        self.device_entry.insert(0, filename)

    def browse_output(self):
        directory = filedialog.askdirectory()
        self.output_entry.delete(0, tk.END)
        self.output_entry.insert(0, directory)

    def start_recovery(self):
        device_path = self.device_entry.get()
        fs_type = self.fs_type_var.get()
        output_dir = self.output_entry.get()

        if not device_path or not output_dir:
            self.status_label.config(text="Please fill in all fields")
            return

        self.progress.start()
        self.start_button.config(state=tk.DISABLED)

        thread = threading.Thread(target=self.run_recovery, args=(device_path, fs_type, output_dir))
        thread.start()

    def run_recovery(self, device_path, fs_type, output_dir):
        try:
            tool = FileSystemRecoveryTool(device_path, fs_type)
            tool.open_device()
            deleted_files = tool.scan_for_deleted_files()
            
            recovered_count = 0
            for file_info in deleted_files:
                if tool.recover_file(file_info, output_dir):
                    recovered_count += 1

            self.master.after(0, self.update_status, f"Recovery complete. Recovered {recovered_count} files.")
        except Exception as e:
            self.master.after(0, self.update_status, f"Error during recovery: {str(e)}")
        finally:
            tool.close_device()
            self.master.after(0, self.recovery_complete)

    def update_status(self, message):
        self.status_label.config(text=message)

    def recovery_complete(self):
        self.progress.stop()
        self.start_button.config(state=tk.NORMAL)

def main():
    root = tk.Tk()
    gui = RecoveryGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

In [1]:
import os
import struct
import argparse
from typing import List, Dict, Any
import ctypes
import mmap
import tkinter as tk
from tkinter import filedialog, ttk
import threading
import logging
import hashlib
import json
from concurrent.futures import ThreadPoolExecutor
import time
import platform

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Expanded file signatures database
FILE_SIGNATURES = {
    b'\xFF\xD8\xFF': 'jpg',
    b'\x89\x50\x4E\x47': 'png',
    b'\x25\x50\x44\x46': 'pdf',
    b'\x50\x4B\x03\x04': 'zip',
    b'\x52\x61\x72\x21': 'rar',
    b'\x47\x49\x46\x38': 'gif',
    b'\x00\x00\x00\x0C\x6A\x50\x20\x20': 'jp2',
    b'\x66\x74\x79\x70\x69\x73\x6F\x6D': 'mp4',
    b'\x1F\x8B\x08': 'gz',
    b'\x42\x5A\x68': 'bz2',
    b'\x37\x7A\xBC\xAF\x27\x1C': '7z',
    b'\x75\x73\x74\x61\x72': 'tar',
    b'\x4F\x67\x67\x53': 'ogg',
    b'\x49\x44\x33': 'mp3',
    b'\x00\x00\x00\x14\x66\x74\x79\x70': 'm4a',
    b'\x4D\x5A': 'exe',
    b'\x7F\x45\x4C\x46': 'elf',
    b'\xCA\xFE\xBA\xBE': 'class',
    b'\x504B0304': 'docx',
    b'\x504B0304': 'xlsx',
    b'\x504B0304': 'pptx',
    b'\x25\x21\x50\x53': 'ps',
    b'\x7B\x5C\x72\x74\x66': 'rtf',
    b'\x3C\x3F\x78\x6D\x6C': 'xml',
    b'\x3C\x68\x74\x6D\x6C': 'html',
    b'\x5B\x49\x6E\x74\x65\x72\x6E\x65\x74\x53\x68\x6F\x72\x74\x63\x75\x74\x5D': 'url',
}

class RawDisk:
    def __init__(self, device_path):
        self.device_path = device_path
        self.file = None
        self.mm = None

    def open(self):
        try:
            if platform.system() == 'Windows':
                self.file = open(self.device_path, 'rb')
                self.mm = mmap.mmap(self.file.fileno(), 0, access=mmap.ACCESS_READ)
            else:
                self.file = open(self.device_path, 'rb')
                self.mm = mmap.mmap(self.file.fileno(), 0, prot=mmap.PROT_READ)
        except IOError as e:
            logger.error(f"Error opening device: {e}")
            raise

    def read(self, offset, size):
        self.mm.seek(offset)
        return self.mm.read(size)

    def close(self):
        if self.mm:
            self.mm.close()
        if self.file:
            self.file.close()

def detect_file_type(data):
    for signature, file_type in FILE_SIGNATURES.items():
        if data.startswith(signature):
            return file_type
    return 'unknown'

class FragmentedFile:
    def __init__(self, start_offset, file_type):
        self.fragments = [(start_offset, None)]  # List of (start, end) offsets
        self.file_type = file_type

    def add_fragment(self, start, end):
        self.fragments.append((start, end))

    def get_size(self):
        return sum(end - start for start, end in self.fragments if end is not None)

class FileSystemRecoveryTool:
    def __init__(self, device_path: str, filesystem_type: str):
        self.device_path = device_path
        self.filesystem_type = filesystem_type
        self.device = None
        self.block_size = 4096  # Default block size, adjust based on filesystem

    def open_device(self):
        try:
            self.device = RawDisk(self.device_path)
            self.device.open()
        except IOError as e:
            logger.error(f"Error opening device: {e}")
            raise

    def close_device(self):
        if self.device:
            self.device.close()

    def read_superblock(self) -> Dict[str, Any]:
        if self.filesystem_type == 'xfs':
            return self.read_xfs_superblock()
        elif self.filesystem_type == 'btrfs':
            return self.read_btrfs_superblock()
        else:
            raise ValueError("Unsupported filesystem type")

    def read_xfs_superblock(self) -> Dict[str, Any]:
        superblock_data = self.device.read(0, 512)
        magic = struct.unpack_from('I', superblock_data, 0)[0]
        if magic != 0x58465342:  # 'XFSB'
            raise ValueError("Invalid XFS superblock")
        block_size = struct.unpack_from('I', superblock_data, 4)[0]
        self.block_size = block_size
        # Extract other relevant information...
        return {'magic': magic, 'block_size': block_size}

    def read_btrfs_superblock(self) -> Dict[str, Any]:
        superblock_data = self.device.read(65536, 4096)
        magic = struct.unpack_from('8s', superblock_data, 64)[0]
        if magic != b'_BHRfS_M':
            raise ValueError("Invalid Btrfs superblock")
        sector_size = struct.unpack_from('I', superblock_data, 108)[0]
        self.block_size = sector_size
        # Extract other relevant information...
        return {'magic': magic, 'sector_size': sector_size}

    def scan_for_deleted_files(self) -> List[Dict[str, Any]]:
        deleted_files = []
        offset = 0
        device_size = os.path.getsize(self.device_path)

        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            futures = []
            while offset < device_size:
                future = executor.submit(self.scan_chunk, offset, min(1024*1024, device_size - offset))
                futures.append(future)
                offset += 1024*1024  # Scan in 1MB chunks

            for future in futures:
                deleted_files.extend(future.result())

        return deleted_files

    def scan_chunk(self, start_offset, size):
        chunk_files = []
        offset = start_offset
        end_offset = start_offset + size
        while offset < end_offset:
            chunk = self.device.read(offset, min(self.block_size, end_offset - offset))
            file_type = detect_file_type(chunk)
            if file_type != 'unknown':
                file_info = self.carve_file(offset, file_type)
                if file_info:
                    chunk_files.append(file_info)
                    offset = file_info['end_offset']
                else:
                    offset += self.block_size
            else:
                offset += self.block_size
        return chunk_files

    def carve_file(self, start_offset: int, file_type: str) -> Dict[str, Any]:
        fragmented_file = FragmentedFile(start_offset, file_type)
        current_offset = start_offset
        max_size = 100 * 1024 * 1024  # 100 MB max file size for this example

        while fragmented_file.get_size() < max_size:
            chunk = self.device.read(current_offset, self.block_size)
            if not chunk:
                break

            # Check for end of file signature (simplified)
            if file_type == 'jpg' and b'\xFF\xD9' in chunk:
                end_offset = current_offset + chunk.index(b'\xFF\xD9') + 2
                fragmented_file.add_fragment(current_offset, end_offset)
                break
            elif file_type == 'png' and b'IEND' in chunk:
                end_offset = current_offset + chunk.index(b'IEND') + 8
                fragmented_file.add_fragment(current_offset, end_offset)
                break
            # Add more end-of-file checks for other file types

            fragmented_file.add_fragment(current_offset, current_offset + len(chunk))
            current_offset += len(chunk)

        if fragmented_file.get_size() > 0:
            return {
                'start_offset': start_offset,
                'end_offset': fragmented_file.fragments[-1][1],
                'type': file_type,
                'fragments': fragmented_file.fragments,
                'size': fragmented_file.get_size()
            }
        return None

    def recover_file(self, file_info: Dict[str, Any], output_dir: str) -> bool:
        try:
            output_path = os.path.join(output_dir, f"recovered_{file_info['start_offset']}.{file_info['type']}")
            with open(output_path, 'wb') as f:
                for start, end in file_info['fragments']:
                    f.write(self.device.read(start, end - start))
            
            # Validate recovered file
            if self.validate_file(output_path, file_info['type']):
                logger.info(f"Successfully recovered and validated file: {output_path}")
                return True
            else:
                logger.warning(f"File recovered but failed validation: {output_path}")
                return False
        except IOError as e:
            logger.error(f"Error recovering file at offset {file_info['start_offset']}: {e}")
            return False

    def validate_file(self, file_path: str, file_type: str) -> bool:
        # Implement file-specific validation here
        # This is a basic example and should be expanded for each file type
        with open(file_path, 'rb') as f:
            header = f.read(8)
            if file_type == 'jpg' and header.startswith(b'\xFF\xD8\xFF'):
                return True
            elif file_type == 'png' and header.startswith(b'\x89\x50\x4E\x47'):
                return True
            # Add more file type validations
        return False

    def extract_metadata(self, file_info: Dict[str, Any]) -> Dict[str, Any]:
        metadata = {
            'start_offset': file_info['start_offset'],
            'end_offset': file_info['end_offset'],
            'size': file_info['size'],
            'type': file_info['type'],
            'fragments': len(file_info['fragments']),
        }

        # Extract filesystem-specific metadata
        if self.filesystem_type == 'xfs':
            metadata.update(self.extract_xfs_metadata(file_info['start_offset']))
        elif self.filesystem_type == 'btrfs':
            metadata.update(self.extract_btrfs_metadata(file_info['start_offset']))

        return metadata

    def extract_xfs_metadata(self, offset: int) -> Dict[str, Any]:
        # Implement XFS-specific metadata extraction
        # This is a placeholder and should be implemented based on XFS structure
        return {"filesystem": "XFS"}

    def extract_btrfs_metadata(self, offset: int) -> Dict[str, Any]:
        # Implement Btrfs-specific metadata extraction
        # This is a placeholder and should be implemented based on Btrfs structure
        return {"filesystem": "Btrfs"}

    def generate_report(self, recovered_files: List[Dict[str, Any]], output_dir: str):
        report = {
            "total_files_recovered": len(recovered_files),
            "filesystem_type": self.filesystem_type,
            "device_path": self.device_path,
            "recovered_files": recovered_files
        }
        report_path = os.path.join(output_dir, "recovery_report.json")
        with open(report_path, 'w') as f:
            json.dump(report, f, indent=2)
        logger.info(f"Recovery report generated: {report_path}")

class RecoveryGUI:
    def __init__(self, master):
        self.master = master
        master.title("Advanced Filesystem Recovery Tool")

        self.device_label = tk.Label(master, text="Device Path:")
        self.device_label.grid(row=0, column=0, sticky="e")
        self.device_entry = tk.Entry(master, width=50)
        self.device_entry.grid(row=0, column=1)
        self.device_button = tk.Button(master, text="Browse", command=self.browse_device)
        self.device_button.grid(row=0, column=2)

        self.fs_type_label = tk.Label(master, text="Filesystem Type:")
        self.fs_type_label.grid(row=1, column=0, sticky="e")
        self.fs_type_var = tk.StringVar(value="xfs")
        self.fs_type_menu = tk.OptionMenu(master, self.fs_type_var, "xfs", "btrfs")
        self.fs_type_menu.grid(row=1, column=1, sticky="w")

        self.output_label = tk.Label(master, text="Output Directory:")
        self.output_label.grid(row=2, column=0, sticky="e")
        self.output_entry = tk.Entry(master, width=50)
        self.output_entry.grid(row=2, column=1)
        self.output_button = tk.Button(master, text="Browse", command=self.browse_output)
        self.output_button.grid(row=2, column=2)

        self.start_button = tk.Button(master, text="Start Recovery", command=self.start_recovery)
        self.start_button.grid(row=3, column=1)

        self.progress = ttk.Progressbar(master, length=300, mode='determinate')
        self.progress.grid(row=4, column=0, columnspan=3, pady=10)

        self.status_label = tk.Label(master, text="")
        self.status_label.grid(row=5, column=0, columnspan=3)

        self.log_text = tk.Text(master, height=10, width=70)
        self.log_text.grid(row=6, column=0, columnspan=3, pady=10)
        self.log_text.config(state=tk.DISABLED)

    def browse_device(self):
        filename = filedialog.askopenfilename()
        self.device_entry.delete(0, tk.END)
        self.device_entry.insert(0, filename)
    def browse_output(self):
        directory = filedialog.askdirectory()
        self.output_entry.delete(0, tk.END)
        self.output_entry.insert(0, directory)

    def start_recovery(self):
        device_path = self.device_entry.get()
        fs_type = self.fs_type_var.get()
        output_dir = self.output_entry.get()

        if not device_path or not output_dir:
            self.update_status("Please fill in all fields")
            return

        self.progress['value'] = 0
        self.start_button.config(state=tk.DISABLED)
        self.clear_log()

        thread = threading.Thread(target=self.run_recovery, args=(device_path, fs_type, output_dir))
        thread.start()

    def run_recovery(self, device_path, fs_type, output_dir):
        try:
            tool = FileSystemRecoveryTool(device_path, fs_type)
            tool.open_device()
            
            self.update_status("Scanning for deleted files...")
            deleted_files = tool.scan_for_deleted_files()
            self.update_progress(50)
            
            self.update_status(f"Found {len(deleted_files)} potentially recoverable files. Starting recovery...")
            recovered_files = []
            for i, file_info in enumerate(deleted_files):
                if tool.recover_file(file_info, output_dir):
                    metadata = tool.extract_metadata(file_info)
                    recovered_files.append(metadata)
                self.update_progress(50 + (i + 1) / len(deleted_files) * 50)

            tool.generate_report(recovered_files, output_dir)
            self.update_status(f"Recovery complete. Recovered {len(recovered_files)} files.")
        except Exception as e:
            self.update_status(f"Error during recovery: {str(e)}")
            logger.exception("Error during recovery process")
        finally:
            tool.close_device()
            self.recovery_complete()

    def update_status(self, message):
        self.status_label.config(text=message)
        self.log_message(message)

    def update_progress(self, value):
        self.progress['value'] = value
        self.master.update_idletasks()

    def recovery_complete(self):
        self.start_button.config(state=tk.NORMAL)

    def log_message(self, message):
        self.log_text.config(state=tk.NORMAL)
        self.log_text.insert(tk.END, message + "\n")
        self.log_text.see(tk.END)
        self.log_text.config(state=tk.DISABLED)

    def clear_log(self):
        self.log_text.config(state=tk.NORMAL)
        self.log_text.delete('1.0', tk.END)
        self.log_text.config(state=tk.DISABLED)

def main():
    root = tk.Tk()
    gui = RecoveryGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
    

2024-09-08 21:37:33,789 - INFO - Recovery report generated: C:/Users/ujjwa/Downloads\recovery_report.json
2024-09-08 21:38:06,310 - INFO - Recovery report generated: C:/Users/ujjwa/Downloads\recovery_report.json
2024-09-08 21:38:07,355 - INFO - Recovery report generated: C:/Users/ujjwa/Downloads\recovery_report.json
2024-09-08 21:38:08,030 - INFO - Recovery report generated: C:/Users/ujjwa/Downloads\recovery_report.json
2024-09-08 21:38:08,619 - INFO - Recovery report generated: C:/Users/ujjwa/Downloads\recovery_report.json
2024-09-08 21:38:11,275 - INFO - Recovery report generated: C:/Users/ujjwa/Downloads\recovery_report.json


In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton, QFileDialog, QTreeWidget, QTreeWidgetItem, QTextEdit
from PyQt5.QtCore import Qt

class FileRecoveryApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("XFS and Btrfs File Recovery")
        self.setGeometry(100, 100, 800, 600)

        main_widget = QWidget()
        self.setCentralWidget(main_widget)

        main_layout = QVBoxLayout()
        main_widget.setLayout(main_layout)

        # Select folder button
        self.select_folder_button = QPushButton("Select Folder")
        self.select_folder_button.clicked.connect(self.select_folder)
        main_layout.addWidget(self.select_folder_button)

        # File tree
        self.file_tree = QTreeWidget()
        self.file_tree.setHeaderLabels(["Filename", "Size", "Date Modified"])
        self.file_tree.itemClicked.connect(self.show_file_details)
        main_layout.addWidget(self.file_tree)

        # File details
        self.file_details = QTextEdit()
        self.file_details.setReadOnly(True)
        main_layout.addWidget(self.file_details)

        # Recover button
        self.recover_button = QPushButton("Recover Selected File")
        self.recover_button.clicked.connect(self.recover_file)
        main_layout.addWidget(self.recover_button)

    def select_folder(self):
        folder = QFileDialog.getExistingDirectory(self, "Select Folder")
        if folder:
            self.scan_folder(folder)

    def scan_folder(self, folder):
        # In a real application, this method would scan the filesystem for deleted files
        # For demonstration, we'll just add some dummy items
        self.file_tree.clear()
        for i in range(5):
            item = QTreeWidgetItem(["Deleted File {}.txt".format(i), "1024 bytes", "2023-09-08"])
            self.file_tree.addTopLevelItem(item)

    def show_file_details(self, item):
        # In a real application, this would show actual file metadata
        filename = item.text(0)
        details = f"Filename: {filename}\nSize: {item.text(1)}\nDate Modified: {item.text(2)}\n"
        details += "This file has been deleted and is available for recovery."
        self.file_details.setText(details)

    def recover_file(self):
        selected_items = self.file_tree.selectedItems()
        if not selected_items:
            return
        
        item = selected_items[0]
        filename = item.text(0)
        # In a real application, this would perform the actual file recovery
        print(f"Recovering file: {filename}")
        # You would add the actual recovery logic here

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FileRecoveryApp()
    window.show()
    sys.exit(app.exec_())

Recovering file: Deleted File 4.txt


SystemExit: 0

c:\Users\ujjwa\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import os
import win32api
import win32file
import winioctlcon
import struct
from datetime import datetime
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton, QFileDialog, QTreeWidget, QTreeWidgetItem, QTextEdit, QMessageBox
from PyQt5.QtCore import Qt, QThread, pyqtSignal

class FilesystemScanner(QThread):
    update_progress = pyqtSignal(str)
    scan_complete = pyqtSignal(list)

    def __init__(self, path):
        super().__init__()
        self.path = path

    def run(self):
        deleted_files = self.scan_ntfs()
        self.scan_complete.emit(deleted_files)

    def scan_ntfs(self):
        deleted_files = []
        self.update_progress.emit("Scanning NTFS filesystem...")
        
        drive_letter = os.path.splitdrive(self.path)[0]
        
        try:
            drive_handle = win32file.CreateFile(
                f"\\\\.\\{drive_letter}",
                win32file.GENERIC_READ,
                win32file.FILE_SHARE_READ | win32file.FILE_SHARE_WRITE,
                None,
                win32file.OPEN_EXISTING,
                0,
                None
            )

            # Read the Master File Table (MFT)
            # This is a simplified approach and would need to be expanded for a real application
            sectors_per_cluster, bytes_per_sector, _, _ = win32file.GetDiskFreeSpace(drive_letter)
            bytes_per_cluster = sectors_per_cluster * bytes_per_sector

            # Read the first few entries of the MFT
            # In a real application, you'd need to read and parse the entire MFT
            mft_data = win32file.DeviceIoControl(
                drive_handle, 
                winioctlcon.FSCTL_GET_NTFS_FILE_RECORD, 
                struct.pack('Q', 0),  # File Reference Number (0 is the MFT itself)
                bytes_per_cluster
            )

            # Parse MFT data to find deleted files
            # This is a placeholder - actual parsing would be much more complex
            # and would require detailed knowledge of the NTFS structure
            deleted_files.append({
                'name': 'Example Deleted File.txt',
                'size': '1024 bytes',
                'modified': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'filesystem': 'NTFS'
            })

            win32file.CloseHandle(drive_handle)

        except Exception as e:
            self.update_progress.emit(f"Error scanning NTFS: {str(e)}")

        self.update_progress.emit("NTFS scan complete")
        return deleted_files

class FileRecoverer(QThread):
    update_progress = pyqtSignal(str)
    recovery_complete = pyqtSignal(bool)

    def __init__(self, file_info, destination):
        super().__init__()
        self.file_info = file_info
        self.destination = destination

    def run(self):
        success = self.recover_ntfs()
        self.recovery_complete.emit(success)

    def recover_ntfs(self):
        self.update_progress.emit(f"Attempting to recover {self.file_info['name']} from NTFS...")
        
        # Placeholder for actual NTFS file recovery logic
        # In a real application, this would involve complex operations
        # to reconstruct the file from the MFT and data runs
        
        self.update_progress.emit("NTFS recovery attempt complete")
        return True  # Placeholder

class FileRecoveryApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Windows NTFS File Recovery")
        self.setGeometry(100, 100, 900, 700)

        main_widget = QWidget()
        self.setCentralWidget(main_widget)

        main_layout = QVBoxLayout()
        main_widget.setLayout(main_layout)

        # Select folder button
        self.select_folder_button = QPushButton("Select Drive")
        self.select_folder_button.clicked.connect(self.select_drive)
        main_layout.addWidget(self.select_folder_button)

        # File tree
        self.file_tree = QTreeWidget()
        self.file_tree.setHeaderLabels(["Filename", "Size", "Date Modified", "Filesystem"])
        self.file_tree.itemClicked.connect(self.show_file_details)
        main_layout.addWidget(self.file_tree)

        # File details
        self.file_details = QTextEdit()
        self.file_details.setReadOnly(True)
        main_layout.addWidget(self.file_details)

        # Recover button
        self.recover_button = QPushButton("Recover Selected File")
        self.recover_button.clicked.connect(self.recover_file)
        main_layout.addWidget(self.recover_button)

        # Progress log
        self.progress_log = QTextEdit()
        self.progress_log.setReadOnly(True)
        main_layout.addWidget(self.progress_log)

    def select_drive(self):
        drive = QFileDialog.getExistingDirectory(self, "Select Drive")
        if drive:
            self.scan_filesystem(drive)

    def scan_filesystem(self, path):
        self.scanner = FilesystemScanner(path)
        self.scanner.update_progress.connect(self.update_progress_log)
        self.scanner.scan_complete.connect(self.update_file_tree)
        self.scanner.start()

    def update_progress_log(self, message):
        self.progress_log.append(message)

    def update_file_tree(self, deleted_files):
        self.file_tree.clear()
        for file in deleted_files:
            item = QTreeWidgetItem([file['name'], file['size'], file['modified'], file['filesystem']])
            self.file_tree.addTopLevelItem(item)

    def show_file_details(self, item):
        filename = item.text(0)
        details = f"Filename: {filename}\n"
        details += f"Size: {item.text(1)}\n"
        details += f"Date Modified: {item.text(2)}\n"
        details += f"Filesystem: {item.text(3)}\n"
        details += "This file has been deleted and is available for recovery."
        self.file_details.setText(details)

    def recover_file(self):
        selected_items = self.file_tree.selectedItems()
        if not selected_items:
            return
        
        item = selected_items[0]
        file_info = {
            'name': item.text(0),
            'size': item.text(1),
            'modified': item.text(2),
            'filesystem': item.text(3)
        }

        destination = QFileDialog.getExistingDirectory(self, "Select Recovery Destination")
        if destination:
            self.recoverer = FileRecoverer(file_info, destination)
            self.recoverer.update_progress.connect(self.update_progress_log)
            self.recoverer.recovery_complete.connect(self.recovery_finished)
            self.recoverer.start()

    def recovery_finished(self, success):
        if success:
            QMessageBox.information(self, "Recovery Complete", "File recovery completed successfully.")
        else:
            QMessageBox.warning(self, "Recovery Failed", "File recovery failed. Please check the logs for details.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FileRecoveryApp()
    window.show()
    sys.exit(app.exec_())

In [1]:
import os
import sys
import winshell
from datetime import datetime
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton, QFileDialog, QTreeWidget, QTreeWidgetItem, QTextEdit, QMessageBox, QProgressBar
from PyQt5.QtCore import Qt, QThread, pyqtSignal
from PyQt5.QtGui import QIcon

def get_file_size(file_path):
    try:
        return os.path.getsize(file_path)
    except Exception:
        return 0

class RecycleBinScanner(QThread):
    update_progress = pyqtSignal(int)
    scan_complete = pyqtSignal(list)

    def run(self):
        deleted_files = []
        recycle_bin = winshell.recycle_bin()
        total_items = len(list(recycle_bin))

        for i, item in enumerate(recycle_bin):
            try:
                file_info = {
                    'name': item.original_filename(),
                    'size': f"{get_file_size(item.original_filename())} bytes",
                    'modified': item.recycle_date().strftime('%Y-%m-%d %H:%M:%S'),
                    'path': item.original_filename(),  # No need to use path here
                }
                deleted_files.append(file_info)
            except Exception as e:
                print(f"Error processing item: {str(e)}")
            self.update_progress.emit(int((i + 1) / total_items * 100))

        self.scan_complete.emit(deleted_files)

class FileRecoverer(QThread):
    update_progress = pyqtSignal(str)
    recovery_complete = pyqtSignal(bool)

    def __init__(self, file_info, destination):
        super().__init__()
        self.file_info = file_info
        self.destination = destination

    def run(self):
        try:
            self.update_progress.emit(f"Recovering {self.file_info['name']}...")
            winshell.undelete(self.file_info['name'])
            recovered_path = os.path.join(self.destination, os.path.basename(self.file_info['name']))
            os.rename(self.file_info['path'], recovered_path)
            self.update_progress.emit(f"File recovered to {recovered_path}")
            self.recovery_complete.emit(True)
        except Exception as e:
            self.update_progress.emit(f"Error during recovery: {str(e)}")
            self.recovery_complete.emit(False)

class FileRecoveryApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Windows Recycle Bin File Recovery")
        self.setGeometry(100, 100, 900, 700)
        self.setWindowIcon(QIcon('recycling.png'))  # Make sure to have this icon file in the same directory

        main_widget = QWidget()
        self.setCentralWidget(main_widget)

        main_layout = QVBoxLayout()
        main_widget.setLayout(main_layout)

        # Scan button
        self.scan_button = QPushButton("Scan Recycle Bin")
        self.scan_button.clicked.connect(self.scan_recycle_bin)
        main_layout.addWidget(self.scan_button)

        # Progress bar
        self.progress_bar = QProgressBar()
        main_layout.addWidget(self.progress_bar)

        # File tree
        self.file_tree = QTreeWidget()
        self.file_tree.setHeaderLabels(["Filename", "Size", "Date Modified"])
        self.file_tree.itemClicked.connect(self.show_file_details)
        main_layout.addWidget(self.file_tree)

        # File details
        self.file_details = QTextEdit()
        self.file_details.setReadOnly(True)
        main_layout.addWidget(self.file_details)

        # Recover button
        self.recover_button = QPushButton("Recover Selected File")
        self.recover_button.clicked.connect(self.recover_file)
        main_layout.addWidget(self.recover_button)

        # Progress log
        self.progress_log = QTextEdit()
        self.progress_log.setReadOnly(True)
        main_layout.addWidget(self.progress_log)

    def scan_recycle_bin(self):
        self.progress_bar.setValue(0)
        self.scanner = RecycleBinScanner()
        self.scanner.update_progress.connect(self.update_progress_bar)
        self.scanner.scan_complete.connect(self.update_file_tree)
        self.scanner.start()

    def update_progress_bar(self, value):
        self.progress_bar.setValue(value)

    def update_file_tree(self, deleted_files):
        self.file_tree.clear()
        for file in deleted_files:
            item = QTreeWidgetItem([file['name'], file['size'], file['modified']])
            item.setData(0, Qt.UserRole, file)
            self.file_tree.addTopLevelItem(item)

    def show_file_details(self, item):
        file_info = item.data(0, Qt.UserRole)
        details = f"Filename: {file_info['name']}\n"
        details += f"Size: {file_info['size']}\n"
        details += f"Date Modified: {file_info['modified']}\n"
        details += f"Original Path: {file_info['path']}\n"
        details += "This file has been deleted and is available for recovery."
        self.file_details.setText(details)

    def recover_file(self):
        selected_items = self.file_tree.selectedItems()
        if not selected_items:
            return

        file_info = selected_items[0].data(0, Qt.UserRole)
        destination = QFileDialog.getExistingDirectory(self, "Select Recovery Destination")
        if destination:
            self.recoverer = FileRecoverer(file_info, destination)
            self.recoverer.update_progress.connect(self.update_progress_log)
            self.recoverer.recovery_complete.connect(self.recovery_finished)
            self.recoverer.start()

    def update_progress_log(self, message):
        self.progress_log.append(message)

    def recovery_finished(self, success):
        if success:
            QMessageBox.information(self, "Recovery Complete", "File recovery completed successfully.")
        else:
            QMessageBox.warning(self, "Recovery Failed", "File recovery failed. Please check the logs for details.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FileRecoveryApp()
    window.show()
    sys.exit(app.exec_())


In [1]:
import os
import sys
import shutil
import winshell
from datetime import datetime
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QLabel, QPushButton, QFileDialog, QTreeWidget, QTreeWidgetItem, QTextEdit, QMessageBox, QProgressBar
from PyQt5.QtCore import Qt, QThread, pyqtSignal
from PyQt5.QtGui import QIcon

def get_file_size(file_path):
    try:
        return os.path.getsize(file_path)
    except:
        return 0

class RecycleBinScanner(QThread):
    update_progress = pyqtSignal(int)
    scan_complete = pyqtSignal(list)

    def run(self):
        deleted_files = []
        recycle_bin = winshell.recycle_bin()
        total_items = len(list(recycle_bin))
        
        for i, item in enumerate(recycle_bin):
            try:
                file_info = {
                    'name': item.original_filename(),
                    'size': f"{get_file_size(item.original_filename())} bytes",
                    'modified': item.recycle_date().strftime('%Y-%m-%d %H:%M:%S'),
                    'path': item.original_filename(),
                    'recycle_bin_path': item.path()
                }
                deleted_files.append(file_info)
            except Exception as e:
                print(f"Error processing item: {str(e)}")
            self.update_progress.emit(int((i + 1) / total_items * 100))

        self.scan_complete.emit(deleted_files)

class FileRecoverer(QThread):
    update_progress = pyqtSignal(str)
    recovery_complete = pyqtSignal(bool)

    def __init__(self, file_info, destination):
        super().__init__()
        self.file_info = file_info
        self.destination = destination

    def run(self):
        try:
            self.update_progress.emit(f"Recovering {self.file_info['name']}...")
            # Attempt to undelete the file
            winshell.undelete(self.file_info['recycle_bin_path'])
            recovered_path = os.path.join(self.destination, os.path.basename(self.file_info['name']))

            # Use shutil.move() to handle cross-drive file movement
            shutil.move(self.file_info['path'], recovered_path)
            self.update_progress.emit(f"File recovered to {recovered_path}")
            self.recovery_complete.emit(True)
        except Exception as e:
            self.update_progress.emit(f"Error during recovery: {str(e)}")
            self.recovery_complete.emit(False)

class FileRecoveryApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Windows Recycle Bin File Recovery")
        self.setGeometry(100, 100, 900, 700)
        self.setWindowIcon(QIcon('recycling.png'))  # Make sure to have this icon file in the same directory

        main_widget = QWidget()
        self.setCentralWidget(main_widget)

        main_layout = QVBoxLayout()
        main_widget.setLayout(main_layout)

        # Scan button
        self.scan_button = QPushButton("Scan Recycle Bin")
        self.scan_button.clicked.connect(self.scan_recycle_bin)
        main_layout.addWidget(self.scan_button)

        # Progress bar
        self.progress_bar = QProgressBar()
        main_layout.addWidget(self.progress_bar)

        # File tree
        self.file_tree = QTreeWidget()
        self.file_tree.setHeaderLabels(["Filename", "Size", "Date Modified"])
        self.file_tree.itemClicked.connect(self.show_file_details)
        main_layout.addWidget(self.file_tree)

        # File details
        self.file_details = QTextEdit()
        self.file_details.setReadOnly(True)
        main_layout.addWidget(self.file_details)

        # Recover button
        self.recover_button = QPushButton("Recover Selected File")
        self.recover_button.clicked.connect(self.recover_file)
        main_layout.addWidget(self.recover_button)

        # Progress log
        self.progress_log = QTextEdit()
        self.progress_log.setReadOnly(True)
        main_layout.addWidget(self.progress_log)

    def scan_recycle_bin(self):
        self.progress_bar.setValue(0)
        self.scanner = RecycleBinScanner()
        self.scanner.update_progress.connect(self.update_progress_bar)
        self.scanner.scan_complete.connect(self.update_file_tree)
        self.scanner.start()

    def update_progress_bar(self, value):
        self.progress_bar.setValue(value)

    def update_file_tree(self, deleted_files):
        self.file_tree.clear()
        for file in deleted_files:
            item = QTreeWidgetItem([file['name'], file['size'], file['modified']])
            item.setData(0, Qt.UserRole, file)
            self.file_tree.addTopLevelItem(item)

    def show_file_details(self, item):
        file_info = item.data(0, Qt.UserRole)
        details = f"Filename: {file_info['name']}\n"
        details += f"Size: {file_info['size']}\n"
        details += f"Date Modified: {file_info['modified']}\n"
        details += f"Original Path: {file_info['path']}\n"
        details += "This file has been deleted and is available for recovery."
        self.file_details.setText(details)

    def recover_file(self):
        selected_items = self.file_tree.selectedItems()
        if not selected_items:
            return
        
        file_info = selected_items[0].data(0, Qt.UserRole)
        destination = QFileDialog.getExistingDirectory(self, "Select Recovery Destination")
        if destination:
            self.recoverer = FileRecoverer(file_info, destination)
            self.recoverer.update_progress.connect(self.update_progress_log)
            self.recoverer.recovery_complete.connect(self.recovery_finished)
            self.recoverer.start()

    def update_progress_log(self, message):
        self.progress_log.append(message)

    def recovery_finished(self, success):
        if success:
            QMessageBox.information(self, "Recovery Complete", "File recovery completed successfully.")
        else:
            QMessageBox.warning(self, "Recovery Failed", "File recovery failed. Please check the logs for details.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = FileRecoveryApp()
    window.show()
    sys.exit(app.exec_())


: 

In [1]:
import os
import time
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from datetime import datetime

class WindowsDeletedFileScanner:
    def __init__(self, master):
        self.master = master
        self.master.title("Windows Deleted File Scanner")
        self.master.geometry("800x600")

        self.scan_path = tk.StringVar()
        self.deleted_files = []

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select Folder to Scan:").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=50, textvariable=self.scan_path).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Browse", command=self.browse_folder).grid(column=2, row=0, sticky=tk.W, pady=5)

        ttk.Button(frame, text="Scan for Deleted Files", command=self.scan_deleted_files).grid(column=0, row=1, columnspan=3, pady=10)

        self.file_tree = ttk.Treeview(frame, columns=('Size', 'Date Modified'), show='headings')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.grid(column=0, row=2, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(2, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=3, row=2, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.recover_files).grid(column=0, row=3, columnspan=3, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=4, columnspan=3, sticky=(tk.W, tk.E), pady=5)

    def browse_folder(self):
        folder_selected = filedialog.askdirectory()
        self.scan_path.set(folder_selected)

    def scan_deleted_files(self):
        folder = self.scan_path.get()
        if not folder:
            messagebox.showerror("Error", "Please select a folder to scan.")
            return

        self.deleted_files = []
        self.file_tree.delete(*self.file_tree.get_children())

        total_files = sum([len(files) for r, d, files in os.walk(folder)])
        self.progress['maximum'] = total_files
        processed_files = 0

        for root, dirs, files in os.walk(folder):
            for name in files:
                processed_files += 1
                self.progress['value'] = processed_files
                self.master.update_idletasks()

                file_path = os.path.join(root, name)
                try:
                    # Check if file is marked for deletion
                    if self.is_marked_for_deletion(file_path):
                        size = os.path.getsize(file_path)
                        mtime = os.path.getmtime(file_path)
                        self.deleted_files.append({
                            'path': file_path,
                            'name': name,
                            'size': size,
                            'mtime': mtime
                        })
                        self.file_tree.insert('', 'end', values=(name, f"{size} bytes", datetime.fromtimestamp(mtime)))
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

        messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")

    def is_marked_for_deletion(self, file_path):
        try:
            # Try to rename the file to itself
            # If it succeeds, the file is not marked for deletion
            # If it fails with a PermissionError, the file is likely marked for deletion
            os.rename(file_path, file_path)
            return False
        except PermissionError:
            return True
        except Exception:
            return False

    def recover_files(self):
        selected_items = self.file_tree.selection()
        if not selected_items:
            messagebox.showinfo("Info", "Please select files to recover.")
            return

        recovery_folder = filedialog.askdirectory(title="Select Recovery Folder")
        if not recovery_folder:
            return

        recovered_count = 0
        for item in selected_items:
            values = self.file_tree.item(item)['values']
            file_name = values[0]
            original_path = next(file['path'] for file in self.deleted_files if file['name'] == file_name)
            
            try:
                # Create a new file name to avoid conflicts
                new_file_name = self.get_unique_filename(recovery_folder, file_name)
                shutil.copy2(original_path, os.path.join(recovery_folder, new_file_name))
                recovered_count += 1
            except Exception as e:
                print(f"Error recovering {file_name}: {e}")

        messagebox.showinfo("Recovery Complete", f"Recovered {recovered_count} files to {recovery_folder}")

    def get_unique_filename(self, folder, filename):
        base, extension = os.path.splitext(filename)
        counter = 1
        new_filename = filename
        while os.path.exists(os.path.join(folder, new_filename)):
            new_filename = f"{base}_{counter}{extension}"
            counter += 1
        return new_filename

if __name__ == "__main__":
    root = tk.Tk()
    app = WindowsDeletedFileScanner(root)
    root.mainloop()

In [1]:
import os
import time
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from datetime import datetime
import threading
import queue
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class WindowsFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Windows File Recovery Tool v2")
        self.master.geometry("900x600")

        self.scan_path = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select Folder to Scan:").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=50, textvariable=self.scan_path).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Browse", command=self.browse_folder).grid(column=2, row=0, sticky=tk.W, pady=5)

        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=0, row=1, columnspan=3, pady=10)

        self.file_tree = ttk.Treeview(frame, columns=('Size', 'Date Modified'), show='headings')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.grid(column=0, row=2, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(2, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=3, row=2, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=3, columnspan=3, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=4, columnspan=3, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=5, columnspan=3, sticky=(tk.W, tk.E), pady=5)

    def browse_folder(self):
        folder_selected = filedialog.askdirectory()
        self.scan_path.set(folder_selected)
        logging.info(f"Selected folder: {folder_selected}")

    def start_scan_thread(self):
        scan_thread = threading.Thread(target=self.scan_deleted_files)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_deleted_files(self):
        folder = self.scan_path.get()
        if not folder:
            self.scan_queue.put("ERROR: No folder selected")
            return

        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())

        total_files = sum([len(files) for r, d, files in os.walk(folder)])
        processed_files = 0

        for root, dirs, files in os.walk(folder):
            for name in files:
                processed_files += 1
                if processed_files % 100 == 0:  # Update progress every 100 files
                    progress_percent = (processed_files / total_files) * 100
                    self.scan_queue.put(f"Scanned {processed_files}/{total_files} files ({progress_percent:.2f}%)")
                    self.master.after(0, self.progress.config, {'value': progress_percent})

                file_path = os.path.join(root, name)
                try:
                    if self.is_marked_for_deletion(file_path):
                        size = os.path.getsize(file_path)
                        mtime = os.path.getmtime(file_path)
                        self.deleted_files.append({
                            'path': file_path,
                            'name': name,
                            'size': size,
                            'mtime': mtime
                        })
                        self.master.after(0, self.file_tree.insert, '', 'end', values=(name, f"{size} bytes", datetime.fromtimestamp(mtime)))
                        logging.info(f"Found deleted file: {file_path}")
                except Exception as e:
                    logging.error(f"Error processing {file_path}: {e}")

        self.scan_queue.put("SCAN_COMPLETE")

    def is_marked_for_deletion(self, file_path):
        try:
            os.rename(file_path, file_path)
            return False
        except PermissionError:
            return True
        except Exception:
            return False

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        selected_items = self.file_tree.selection()
        if not selected_items:
            self.recovery_queue.put("ERROR: No files selected for recovery")
            return

        recovery_folder = filedialog.askdirectory(title="Select Recovery Folder")
        if not recovery_folder:
            self.recovery_queue.put("ERROR: No recovery folder selected")
            return

        recovered_count = 0
        total_files = len(selected_items)

        for item in selected_items:
            values = self.file_tree.item(item)['values']
            file_name = values[0]
            original_path = next(file['path'] for file in self.deleted_files if file['name'] == file_name)
            
            try:
                new_file_name = self.get_unique_filename(recovery_folder, file_name)
                shutil.copy2(original_path, os.path.join(recovery_folder, new_file_name))
                recovered_count += 1
                progress_percent = (recovered_count / total_files) * 100
                self.recovery_queue.put(f"Recovered {recovered_count}/{total_files} files ({progress_percent:.2f}%)")
                self.master.after(0, self.progress.config, {'value': progress_percent})
                logging.info(f"Recovered file: {file_name} to {new_file_name}")
            except Exception as e:
                logging.error(f"Error recovering {file_name}: {e}")

        self.recovery_queue.put("RECOVERY_COMPLETE")
        messagebox.showinfo("Recovery Complete", f"Recovered {recovered_count} files to {recovery_folder}")

    def get_unique_filename(self, folder, filename):
        base, extension = os.path.splitext(filename)
        counter = 1
        new_filename = filename
        while os.path.exists(os.path.join(folder, new_filename)):
            new_filename = f"{base}_{counter}{extension}"
            counter += 1
        return new_filename

if __name__ == "__main__":
    root = tk.Tk()
    app = WindowsFileRecoveryTool(root)
    root.mainloop()

2024-09-08 22:26:23,046 - INFO - Selected folder: E:/All In one/DSA
2024-09-08 22:27:19,311 - INFO - Selected folder: E:/certificates online


In [6]:
import os
import struct
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import win32file
import win32con
import winioctlcon
import logging
from datetime import datetime, timedelta
import threading
import queue
import ctypes
import re
import hashlib
import csv
import pywintypes

logging.basicConfig(filename='ntfs_recovery.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

class NTFSFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Advanced NTFS File Recovery Tool")
        self.master.geometry("1200x800")

        self.drive_letter = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select NTFS Drive (e.g., C:):").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=5, textvariable=self.drive_letter).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=2, row=0, sticky=tk.W, pady=5)

        self.file_tree = ttk.Treeview(frame, columns=('Size', 'Date Modified', 'MFT Record'), show='headings')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.heading('MFT Record', text='MFT Record')
        self.file_tree.grid(column=0, row=1, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=3, row=1, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=2, columnspan=3, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=3, columnspan=3, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=4, columnspan=3, sticky=(tk.W, tk.E), pady=5)

    def start_scan_thread(self):
        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())
        self.drive = self.drive_letter.get().upper()
        
        if not self.drive or len(self.drive) != 2 or self.drive[1] != ':':
            messagebox.showerror("Error", "Invalid drive letter")
            return

        scan_thread = threading.Thread(target=self.scan_ntfs)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_ntfs(self):
        try:
            volume_handle = self.open_volume_with_elevated_privileges()

            boot_sector = win32file.ReadFile(volume_handle, 512)[1]
            bytes_per_sector = struct.unpack_from("<H", boot_sector, 0x0B)[0]
            sectors_per_cluster = struct.unpack_from("<B", boot_sector, 0x0D)[0]
            mft_cluster = struct.unpack_from("<Q", boot_sector, 0x30)[0]
            mft_record_size = 1024  # Typically 1024 bytes for NTFS

            mft_offset = mft_cluster * sectors_per_cluster * bytes_per_sector

            win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
            
            for i in range(100000):  # Limit to first 100,000 MFT entries for this example
                try:
                    mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]
                    self.parse_mft_record(mft_record, i)
                except pywintypes.error as e:
                    if e.winerror == 23:  # ERROR_CRC
                        logging.warning(f"CRC error reading MFT record {i}, skipping...")
                        continue
                    else:
                        raise
                
                if i % 1000 == 0:
                    self.scan_queue.put(f"Scanned {i} MFT entries...")

            win32file.CloseHandle(volume_handle)
            self.scan_queue.put("SCAN_COMPLETE")
        except Exception as e:
            error_msg = f"Error scanning NTFS: {str(e)}"
            logging.error(error_msg)
            self.scan_queue.put(f"ERROR: {error_msg}")

    def open_volume_with_elevated_privileges(self):
        try:
            return win32file.CreateFile(
                f"\\\\.\\{self.drive}",
                win32con.GENERIC_READ,
                win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                None,
                win32con.OPEN_EXISTING,
                win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                None
            )
        except pywintypes.error as e:
            if e.winerror == 5:  # Access denied
                logging.warning("Access denied, attempting to enable SeBackupPrivilege...")
                if self.enable_backup_privilege():
                    return win32file.CreateFile(
                        f"\\\\.\\{self.drive}",
                        win32con.GENERIC_READ,
                        win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                        None,
                        win32con.OPEN_EXISTING,
                        win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                        None
                    )
                else:
                    raise ValueError("Failed to enable SeBackupPrivilege. Please run as administrator.")
            else:
                raise

    def enable_backup_privilege(self):
        try:
            import win32security
            import ntsecuritycon

            h_token = win32security.OpenProcessToken(win32process.GetCurrentProcess(), win32security.TOKEN_ALL_ACCESS)
            win32security.AdjustTokenPrivileges(
                h_token, False, [(win32security.LookupPrivilegeValue(None, "SeBackupPrivilege"), win32security.SE_PRIVILEGE_ENABLED)]
            )
            return True
        except Exception as e:
            logging.error(f"Failed to enable SeBackupPrivilege: {str(e)}")
            return False

    def parse_mft_record(self, mft_record, record_number):
        if mft_record[0:4] != b'FILE':
            return

        flags = struct.unpack_from("<H", mft_record, 0x16)[0]
        if flags & 0x01 == 0:  # File is not in use (deleted)
            try:
                first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
                current_offset = first_attr_offset

                file_name = None
                file_size = 0
                mtime = None

                while current_offset < len(mft_record) - 4:
                    attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
                    attr_length = struct.unpack_from("<I", mft_record, current_offset + 4)[0]

                    if attr_type == 0xFFFFFFFF:
                        break

                    if attr_type == 0x30:  # Filename attribute
                        name_length = struct.unpack_from("<B", mft_record, current_offset + 0x58)[0]
                        file_name = mft_record[current_offset + 0x5A:current_offset + 0x5A + name_length * 2].decode('utf-16le')
                        mtime = self.filetime_to_datetime(struct.unpack_from("<Q", mft_record, current_offset + 0x20)[0])
                    elif attr_type == 0x80:  # Data attribute
                        file_size = struct.unpack_from("<Q", mft_record, current_offset + 0x30)[0]

                    current_offset += attr_length

                if file_name and mtime:
                    self.add_deleted_file(file_name, file_size, mtime, record_number)

            except Exception as e:
                logging.error(f"Error parsing MFT record {record_number}: {str(e)}")

    def add_deleted_file(self, name, size, mtime, record_number):
        self.deleted_files.append({
            'name': name,
            'size': size,
            'mtime': mtime,
            'record_number': record_number
        })
        self.master.after(0, self._update_file_tree, name, size, mtime, record_number)
        logging.info(f"Found deleted file: {name} (Record: {record_number})")

    def _update_file_tree(self, name, size, mtime, record_number):
        self.file_tree.insert('', 'end', values=(name, f"{size:,} bytes", mtime, record_number), tags=(str(record_number),))

    @staticmethod
    def filetime_to_datetime(filetime):
        return datetime(1601, 1, 1) + timedelta(microseconds=filetime // 10)

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
                self.progress['value'] = 100
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                progress = float(message.split('%')[0].split('(')[1])
                self.progress['value'] = progress
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        selected_items = self.file_tree.selection()
        if not selected_items:
            error_msg = "No files selected for recovery"
            self.recovery_queue.put(f"ERROR: {error_msg}")
            logging.error(error_msg)
            return

        recovery_folder = filedialog.askdirectory(title="Select Recovery Folder")
        if not recovery_folder:
            error_msg = "No recovery folder selected"
            self.recovery_queue.put(f"ERROR: {error_msg}")
            logging.error(error_msg)
            return

        volume_handle = self.open_volume_with_elevated_privileges()

        try:
            recovered_count = 0
            total_files = len(selected_items)

            for item in selected_items:
                file_info = self.file_tree.item(item)['values']
                file_name, size_str, mtime_str, record_number = file_info
                record_number = int(record_number)

                try:
                    self.recover_file(volume_handle, record_number, file_name, recovery_folder)
                    recovered_count += 1
                    progress_percent = (recovered_count / total_files) * 100
                    self.recovery_queue.put(f"Recovered {recovered_count}/{total_files} files ({progress_percent:.2f}%)")
                except Exception as e:
                    error_msg = f"Error recovering {file_name}: {str(e)}"
                    logging.error(error_msg)
                    self.recovery_queue.put(f"ERROR: {error_msg}")

            self.recovery_queue.put("RECOVERY_COMPLETE")
            self.generate_recovery_report(recovery_folder)
            completion_msg = f"Recovered {recovered_count} files to {recovery_folder}"
            logging.info(completion_msg)
            messagebox.showinfo("Recovery Complete", completion_msg)
        finally:
            win32file.CloseHandle(volume_handle)

    def recover_file(self, volume_handle, record_number, file_name, recovery_folder):
        mft_record_size = 1024  # Assuming 1024-byte MFT records
        mft_offset = record_number * mft_record_size

        win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
        mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]

        if mft_record[0:4] != b'FILE':
            raise ValueError(f"Invalid MFT record for file: {file_name}")

        data_runs = self.get_data_runs(mft_record)
        if not data_runs:
            raise ValueError(f"No data runs found for file: {file_name}")

        recovered_file_path = os.path.join(recovery_folder, self.get_unique_filename(recovery_folder, file_name))
        with open(recovered_file_path, 'wb') as recovered_file:
            for cluster, run_length in data_runs:
                win32file.SetFilePointer(volume_handle, cluster * self.bytes_per_cluster, win32file.FILE_BEGIN)
                data = win32file.ReadFile(volume_handle, run_length * self.bytes_per_cluster)[1]
                recovered_file.write(data)

        self.set_file_times(recovered_file_path, mft_record)
        file_hash = self.calculate_file_hash(recovered_file_path)
        logging.info(f"Recovered file: {file_name} to {recovered_file_path} (SHA-256: {file_hash})")
        return recovered_file_path, file_hash

    def get_data_runs(self, mft_record):
        first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
        current_offset = first_attr_offset

        while current_offset < len(mft_record) - 4:
            attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
            attr_length = struct.unpack_from("<I", mft_record, current_offset + 4)[0]

            if attr_type == 0xFFFFFFFF:
                break

            if attr_type == 0x80:  # Data attribute
                return self.parse_data_runs(mft_record, current_offset)

            current_offset += attr_length

        return []

    def parse_data_runs(self, mft_record, attr_offset):
        data_runs = []
        data_run_offset = attr_offset + struct.unpack_from("<H", mft_record, attr_offset + 0x20)[0]

        cluster_offset = 0
        while mft_record[data_run_offset] != 0:
            run_header = mft_record[data_run_offset]
            run_offset_size = run_header >> 4
            run_length_size = run_header & 0x0F

            data_run_offset += 1
            run_length = int.from_bytes(mft_record[data_run_offset:data_run_offset + run_length_size], 'little')
            data_run_offset += run_length_size

            if run_offset_size > 0:
                run_offset = int.from_bytes(mft_record[data_run_offset:data_run_offset + run_offset_size], 'little', signed=True)
                data_run_offset += run_offset_size
                cluster_offset += run_offset
            else:
                run_offset = 0

            data_runs.append((cluster_offset, run_length))

        return data_runs

    def set_file_times(self, file_path, mft_record):
        created_time = self.extract_filetime(mft_record, 0x10)
        modified_time = self.extract_filetime(mft_record, 0x18)
        accessed_time = self.extract_filetime(mft_record, 0x20)

        win32file.SetFileTime(
            win32file.CreateFile(
                file_path,
                win32con.GENERIC_WRITE,
                0,
                None,
                win32con.OPEN_EXISTING,
                0,
                None
            ),
            created_time,
            accessed_time,
            modified_time
        )

    def extract_filetime(self, mft_record, offset):
        filetime = struct.unpack_from("<Q", mft_record, offset)[0]
        return self.filetime_to_datetime(filetime)

    @staticmethod
    def calculate_file_hash(file_path):
        sha256_hash = hashlib.sha256()
        with open(file_path, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()

    def get_unique_filename(self, folder, filename):
        base, extension = os.path.splitext(filename)
        counter = 1
        new_filename = filename
        while os.path.exists(os.path.join(folder, new_filename)):
            new_filename = f"{base}_{counter}{extension}"
            counter += 1
        return new_filename

    def generate_recovery_report(self, recovery_folder):
        report_path = os.path.join(recovery_folder, "recovery_report.csv")
        with open(report_path, 'w', newline='') as csvfile:
            fieldnames = ["Original Name", "Recovered Name", "Size", "SHA-256 Hash", "Creation Time", "Modified Time", "Access Time"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for root, _, files in os.walk(recovery_folder):
                for file in files:
                    if file != "recovery_report.csv":
                        file_path = os.path.join(root, file)
                        file_hash = self.calculate_file_hash(file_path)
                        file_size = os.path.getsize(file_path)
                        creation_time, access_time, modify_time = self.get_file_times(file_path)

                        writer.writerow({
                            "Original Name": self.get_original_filename(file),
                            "Recovered Name": file,
                            "Size": file_size,
                            "SHA-256 Hash": file_hash,
                            "Creation Time": creation_time,
                            "Modified Time": modify_time,
                            "Access Time": access_time
                        })

        logging.info(f"Generated recovery report: {report_path}")

    def get_file_times(self, file_path):
        creation_time = os.path.getctime(file_path)
        access_time = os.path.getatime(file_path)
        modify_time = os.path.getmtime(file_path)
        return (
            datetime.fromtimestamp(creation_time).isoformat(),
            datetime.fromtimestamp(access_time).isoformat(),
            datetime.fromtimestamp(modify_time).isoformat()
        )

    def get_original_filename(self, recovered_filename):
        # Remove the _1, _2, etc. suffixes added to avoid filename conflicts
        return re.sub(r'_\d+(\.[^.]+)$', r'\1', recovered_filename)

    def analyze_file_type(self, file_path):
        # This method can be expanded to include more file signatures
        file_signatures = {
            b'\xFF\xD8\xFF': 'JPEG image',
            b'\x89PNG\r\n\x1A\n': 'PNG image',
            b'%PDF': 'PDF document',
            b'PK\x03\x04': 'ZIP archive',
            b'\x50\x4B\x05\x06': 'ZIP archive (empty)',
            b'\x50\x4B\x07\x08': 'ZIP archive (spanned)',
            b'Rar!\x1A\x07\x00': 'RAR archive',
            b'Rar!\x1A\x07\x01\x00': 'RAR archive',
            b'\x37\x7A\xBC\xAF\x27\x1C': '7-Zip archive',
        }

        with open(file_path, 'rb') as file:
            file_start = file.read(max(len(sig) for sig in file_signatures))

        for signature, file_type in file_signatures.items():
            if file_start.startswith(signature):
                return file_type

        return "Unknown"

    def deep_scan(self):
        # Implement a more thorough scan that checks for file signatures in unallocated space
        pass

    def carve_files(self):
        # Implement file carving for specific file types
        pass

def main():
    logging.info("Starting Advanced NTFS File Recovery Tool")
    root = tk.Tk()
    app = NTFSFileRecoveryTool(root)
    root.mainloop()
    logging.info("Advanced NTFS File Recovery Tool closed")

if __name__ == "__main__":
    main()

2024-09-08 22:46:46,840 - INFO - Starting Advanced NTFS File Recovery Tool
2024-09-08 22:46:51,327 - WARNING - Access denied, attempting to enable SeBackupPrivilege...
2024-09-08 22:46:51,333 - ERROR - Failed to enable SeBackupPrivilege: name 'win32process' is not defined
2024-09-08 22:46:51,333 - ERROR - Error scanning NTFS: Failed to enable SeBackupPrivilege. Please run as administrator.


In [3]:
import os
import struct
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import win32file
import win32con
import winioctlcon
import logging
from datetime import datetime, timedelta
import threading
import queue
import ctypes
import re
import hashlib
import csv
import pywintypes
import win32security
import win32process

logging.basicConfig(filename='ntfs_recovery.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

class NTFSFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Advanced NTFS File Recovery Tool")
        self.master.geometry("1200x800")

        self.drive_letter = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select NTFS Drive (e.g., C:):").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=5, textvariable=self.drive_letter).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=2, row=0, sticky=tk.W, pady=5)

        self.file_tree = ttk.Treeview(frame, columns=('Name', 'Size', 'Date Modified', 'MFT Record'), show='headings')
        self.file_tree.heading('Name', text='Name')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.heading('MFT Record', text='MFT Record')
        self.file_tree.grid(column=0, row=1, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=3, row=1, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=2, columnspan=3, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=3, columnspan=3, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=4, columnspan=3, sticky=(tk.W, tk.E), pady=5)

    def start_scan_thread(self):
        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())
        self.drive = self.drive_letter.get().upper()
        
        if not self.drive or len(self.drive) != 2 or self.drive[1] != ':':
            messagebox.showerror("Error", "Invalid drive letter")
            return

        scan_thread = threading.Thread(target=self.scan_ntfs)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_ntfs(self):
        try:
            volume_handle = self.open_volume_with_elevated_privileges()

            boot_sector = win32file.ReadFile(volume_handle, 512)[1]
            bytes_per_sector = struct.unpack_from("<H", boot_sector, 0x0B)[0]
            sectors_per_cluster = struct.unpack_from("<B", boot_sector, 0x0D)[0]
            mft_cluster = struct.unpack_from("<Q", boot_sector, 0x30)[0]
            mft_record_size = 1024  # Typically 1024 bytes for NTFS

            mft_offset = mft_cluster * sectors_per_cluster * bytes_per_sector

            win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
            
            for i in range(100000):  # Limit to first 100,000 MFT entries for this example
                try:
                    mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]
                    self.parse_mft_record(mft_record, i)
                except pywintypes.error as e:
                    if e.winerror == 23:  # ERROR_CRC
                        logging.warning(f"CRC error reading MFT record {i}, skipping...")
                        continue
                    else:
                        raise
                
                if i % 1000 == 0:
                    self.scan_queue.put(f"Scanned {i} MFT entries...")

            win32file.CloseHandle(volume_handle)
            self.scan_queue.put("SCAN_COMPLETE")
        except Exception as e:
            error_msg = f"Error scanning NTFS: {str(e)}"
            logging.error(error_msg)
            self.scan_queue.put(f"ERROR: {error_msg}")

    def open_volume_with_elevated_privileges(self):
        try:
            return win32file.CreateFile(
                f"\\\\.\\{self.drive}",
                win32con.GENERIC_READ,
                win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                None,
                win32con.OPEN_EXISTING,
                win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                None
            )
        except pywintypes.error as e:
            if e.winerror == 5:  # Access denied
                logging.warning("Access denied, attempting to enable SeBackupPrivilege...")
                if self.enable_backup_privilege():
                    return win32file.CreateFile(
                        f"\\\\.\\{self.drive}",
                        win32con.GENERIC_READ,
                        win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                        None,
                        win32con.OPEN_EXISTING,
                        win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                        None
                    )
                else:
                    raise ValueError("Failed to enable SeBackupPrivilege. Please run as administrator.")
            else:
                raise

    def enable_backup_privilege(self):
        try:
            h_token = win32security.OpenProcessToken(win32process.GetCurrentProcess(), win32security.TOKEN_ALL_ACCESS)
            win32security.AdjustTokenPrivileges(
                h_token, False, [(win32security.LookupPrivilegeValue(None, "SeBackupPrivilege"), win32security.SE_PRIVILEGE_ENABLED)]
            )
            return True
        except Exception as e:
            logging.error(f"Failed to enable SeBackupPrivilege: {str(e)}")
            return False

    def parse_mft_record(self, mft_record, record_number):
        if mft_record[0:4] != b'FILE':
            return

        flags = struct.unpack_from("<H", mft_record, 0x16)[0]
        if flags & 0x01 == 0:  # File is not in use (deleted)
            try:
                first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
                current_offset = first_attr_offset

                file_name = None
                file_size = 0
                mtime = None

                while current_offset < len(mft_record) - 4:
                    attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
                    attr_length = struct.unpack_from("<I", mft_record, current_offset + 4)[0]

                    if attr_type == 0xFFFFFFFF:
                        break

                    if attr_type == 0x30:  # Filename attribute
                        name_length = struct.unpack_from("<B", mft_record, current_offset + 0x58)[0]
                        file_name = mft_record[current_offset + 0x5A:current_offset + 0x5A + name_length * 2].decode('utf-16le')
                        mtime = self.filetime_to_datetime(struct.unpack_from("<Q", mft_record, current_offset + 0x20)[0])
                    elif attr_type == 0x80:  # Data attribute
                        file_size = struct.unpack_from("<Q", mft_record, current_offset + 0x30)[0]

                    current_offset += attr_length

                if file_name and mtime:
                    self.add_deleted_file(file_name, file_size, mtime, record_number)
            except Exception as e:
                logging.error(f"Failed to parse MFT record {record_number}: {str(e)}")

    def filetime_to_datetime(self, filetime):
        return datetime(1601, 1, 1) + timedelta(microseconds=filetime / 10)

    def add_deleted_file(self, name, size, mtime, record_number):
        self.deleted_files.append({
            'Name': name,
            'Size': size,
            'Date Modified': mtime,
            'MFT Record': record_number
        })
        self.master.after(0, self.update_file_tree)

    def update_file_tree(self):
        for i in self.file_tree.get_children():
            self.file_tree.delete(i)
        for file_info in self.deleted_files:
            self.file_tree.insert('', 'end', values=(file_info['Name'], file_info['Size'], file_info['Date Modified'], file_info['MFT Record']))

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
                messagebox.showinfo("Recovery Complete", "File recovery process is finished.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        try:
            volume_handle = self.open_volume_with_elevated_privileges()

            for file_info in self.deleted_files:
                file_name = file_info['Name']
                mft_record = file_info['MFT Record']

                # Here you would implement actual file recovery logic.
                # For now, we'll just log the operation.
                logging.info(f"Recovering file {file_name} with MFT record {mft_record}...")
                self.recovery_queue.put(f"Recovering {file_name}...")

            self.recovery_queue.put("RECOVERY_COMPLETE")
        except Exception as e:
            error_msg = f"Error recovering files: {str(e)}"
            logging.error(error_msg)
            self.recovery_queue.put(f"ERROR: {error_msg}")

if __name__ == "__main__":
    root = tk.Tk()
    app = NTFSFileRecoveryTool(root)
    root.mainloop()


In [7]:
import os
import struct
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import win32file
import win32con
import winioctlcon
import logging
from datetime import datetime, timedelta
import threading
import queue
import ctypes
import re
import hashlib
import csv
import pywintypes
import win32security
import win32process

logging.basicConfig(filename='ntfs_recovery.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

class NTFSFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Advanced NTFS File Recovery Tool")
        self.master.geometry("1200x800")

        self.drive_letter = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select NTFS Drive (e.g., C:):").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=5, textvariable=self.drive_letter).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=2, row=0, sticky=tk.W, pady=5)

        self.file_tree = ttk.Treeview(frame, columns=('Name', 'Size', 'Date Modified', 'MFT Record'), show='headings')
        self.file_tree.heading('Name', text='Name')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.heading('MFT Record', text='MFT Record')
        self.file_tree.grid(column=0, row=1, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=3, row=1, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=2, columnspan=3, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=3, columnspan=3, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=4, columnspan=3, sticky=(tk.W, tk.E), pady=5)

    def start_scan_thread(self):
        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())
        self.drive = self.drive_letter.get().upper()
        
        if not self.drive or len(self.drive) != 2 or self.drive[1] != ':':
            messagebox.showerror("Error", "Invalid drive letter")
            return

        scan_thread = threading.Thread(target=self.scan_ntfs)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_ntfs(self):
        try:
            volume_handle = self.open_volume_with_elevated_privileges()
            boot_sector = win32file.ReadFile(volume_handle, 512)[1]
            bytes_per_sector = struct.unpack_from("<H", boot_sector, 0x0B)[0]
            sectors_per_cluster = struct.unpack_from("<B", boot_sector, 0x0D)[0]
            mft_cluster = struct.unpack_from("<Q", boot_sector, 0x30)[0]
            mft_record_size = 1024  # Typically 1024 bytes for NTFS

            mft_offset = mft_cluster * sectors_per_cluster * bytes_per_sector

            win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
            
            for i in range(100000):  # Limit to first 100,000 MFT entries for this example
                try:
                    mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]
                    self.parse_mft_record(mft_record, i)
                except pywintypes.error as e:
                    if e.winerror == 23:  # ERROR_CRC
                        logging.warning(f"CRC error reading MFT record {i}, skipping...")
                        continue
                    else:
                        raise
                
                if i % 1000 == 0:
                    self.scan_queue.put(f"Scanned {i} MFT entries...")

            win32file.CloseHandle(volume_handle)
            self.scan_queue.put("SCAN_COMPLETE")
        except Exception as e:
            error_msg = f"Error scanning NTFS: {str(e)}"
            logging.error(error_msg)
            self.scan_queue.put(f"ERROR: {error_msg}")

    def open_volume_with_elevated_privileges(self):
        try:
            handle = win32file.CreateFile(
                f"\\\\.\\{self.drive}",
                win32con.GENERIC_READ,
                win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                None,
                win32con.OPEN_EXISTING,
                win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                None
            )
            return handle
        except pywintypes.error as e:
            if e.winerror == 5:  # Access denied
                logging.warning("Access denied, attempting to enable SeBackupPrivilege...")
                if self.enable_backup_privilege():
                    return win32file.CreateFile(
                        f"\\\\.\\{self.drive}",
                        win32con.GENERIC_READ,
                        win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                        None,
                        win32con.OPEN_EXISTING,
                        win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                        None
                    )
                else:
                    raise ValueError("Failed to enable SeBackupPrivilege. Please run as administrator.")
            else:
                raise

    def enable_backup_privilege(self):
        try:
            h_token = win32security.OpenProcessToken(win32process.GetCurrentProcess(), win32security.TOKEN_ADJUST_PRIVILEGES | win32security.TOKEN_QUERY)
            privs = [(win32security.LookupPrivilegeValue(None, "SeBackupPrivilege"), win32security.SE_PRIVILEGE_ENABLED)]
            win32security.AdjustTokenPrivileges(h_token, False, privs)
            return True
        except Exception as e:
            logging.error(f"Failed to enable SeBackupPrivilege: {str(e)}")
            return False

    def parse_mft_record(self, mft_record, record_number):
        if mft_record[0:4] != b'FILE':
            return

        flags = struct.unpack_from("<H", mft_record, 0x16)[0]
        if flags & 0x01 == 0:  # File is not in use (deleted)
            try:
                first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
                current_offset = first_attr_offset

                file_name = None
                file_size = 0
                mtime = None

                while current_offset < len(mft_record) - 4:
                    attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
                    if attr_type == 0x10:  # $STANDARD_INFORMATION
                        attr_length = struct.unpack_from("<H", mft_record, current_offset + 0x04)[0]
                        file_size = struct.unpack_from("<Q", mft_record, current_offset + 0x08)[0]
                        mtime = self.filetime_to_datetime(struct.unpack_from("<Q", mft_record, current_offset + 0x10)[0])
                    elif attr_type == 0x30:  # $FILE_NAME
                        attr_length = struct.unpack_from("<H", mft_record, current_offset + 0x04)[0]
                        file_name_offset = struct.unpack_from("<H", mft_record, current_offset + 0x20)[0]
                        file_name = mft_record[current_offset + file_name_offset:].decode('utf-16le').split('\x00')[0]

                    current_offset += attr_length

                if file_name:
                    self.add_deleted_file(file_name, file_size, mtime, record_number)
            except Exception as e:
                logging.error(f"Failed to parse MFT record {record_number}: {str(e)}")

    def filetime_to_datetime(self, filetime):
        return datetime(1601, 1, 1) + timedelta(microseconds=filetime / 10)

    def add_deleted_file(self, name, size, mtime, record_number):
        self.deleted_files.append({
            'Name': name,
            'Size': size,
            'Date Modified': mtime,
            'MFT Record': record_number
        })
        self.master.after(0, self.update_file_tree)

    def update_file_tree(self):
        for i in self.file_tree.get_children():
            self.file_tree.delete(i)
        for file_info in self.deleted_files:
            self.file_tree.insert('', 'end', values=(file_info['Name'], file_info['Size'], file_info['Date Modified'], file_info['MFT Record']))

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
                messagebox.showinfo("Recovery Complete", "File recovery process is finished.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        try:
            volume_handle = self.open_volume_with_elevated_privileges()

            for file_info in self.deleted_files:
                file_name = file_info['Name']
                mft_record = file_info['MFT Record']

                # Here you would implement actual file recovery logic.
                # For now, we'll just log the operation.
                logging.info(f"Recovering file {file_name} with MFT record {mft_record}...")
                self.recovery_queue.put(f"Recovering {file_name}...")

            self.recovery_queue.put("RECOVERY_COMPLETE")
        except Exception as e:
            error_msg = f"Error recovering files: {str(e)}"
            logging.error(error_msg)
            self.recovery_queue.put(f"ERROR: {error_msg}")

if __name__ == "__main__":
    root = tk.Tk()
    app = NTFSFileRecoveryTool(root)
    root.mainloop()


In [1]:
import os
import struct
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import win32file
import win32con
import winioctlcon
import logging
from datetime import datetime, timedelta
import threading
import queue
import ctypes
import re
import hashlib
import csv
import pywintypes
import win32security
import win32process

logging.basicConfig(filename='ntfs_recovery.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

class NTFSFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Advanced NTFS File Recovery Tool")
        self.master.geometry("1200x800")

        self.drive_letter = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select NTFS Drive (e.g., C:):").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=5, textvariable=self.drive_letter).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=2, row=0, sticky=tk.W, pady=5)

        self.file_tree = ttk.Treeview(frame, columns=('Name', 'Size', 'Date Modified', 'MFT Record'), show='headings')
        self.file_tree.heading('Name', text='Name')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.heading('MFT Record', text='MFT Record')
        self.file_tree.grid(column=0, row=1, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=3, row=1, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=2, columnspan=3, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=3, columnspan=3, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=4, columnspan=3, sticky=(tk.W, tk.E), pady=5)

    def start_scan_thread(self):
        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())
        self.drive = self.drive_letter.get().upper()
        
        if not self.drive or len(self.drive) != 2 or self.drive[1] != ':':
            messagebox.showerror("Error", "Invalid drive letter")
            return

        scan_thread = threading.Thread(target=self.scan_ntfs)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_ntfs(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.scan_queue.put("ERROR: Failed to open volume. Please run as administrator.")
                return

            boot_sector = win32file.ReadFile(volume_handle, 512)[1]
            bytes_per_sector = struct.unpack_from("<H", boot_sector, 0x0B)[0]
            sectors_per_cluster = struct.unpack_from("<B", boot_sector, 0x0D)[0]
            mft_cluster = struct.unpack_from("<Q", boot_sector, 0x30)[0]
            mft_record_size = 1024  # Typically 1024 bytes for NTFS

            mft_offset = mft_cluster * sectors_per_cluster * bytes_per_sector

            win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
            
            for i in range(100000):  # Limit to first 100,000 MFT entries for this example
                try:
                    mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]
                    self.parse_mft_record(mft_record, i)
                except pywintypes.error as e:
                    if e.winerror == 23:  # ERROR_CRC
                        logging.warning(f"CRC error reading MFT record {i}, skipping...")
                        continue
                    else:
                        raise
                
                if i % 1000 == 0:
                    self.scan_queue.put(f"Scanned {i} MFT entries...")

            win32file.CloseHandle(volume_handle)
            self.scan_queue.put("SCAN_COMPLETE")
        except Exception as e:
            error_msg = f"Error scanning NTFS: {str(e)}"
            logging.error(error_msg)
            self.scan_queue.put(f"ERROR: {error_msg}")

    def open_volume(self):
        try:
            return win32file.CreateFile(
                f"\\\\.\\{self.drive}",
                win32con.GENERIC_READ,
                win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                None,
                win32con.OPEN_EXISTING,
                win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                None
            )
        except pywintypes.error as e:
            if e.winerror == 5:  # Access denied
                logging.warning("Access denied, attempting to enable SeBackupPrivilege...")
                if self.enable_backup_privilege():
                    try:
                        return win32file.CreateFile(
                            f"\\\\.\\{self.drive}",
                            win32con.GENERIC_READ,
                            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                            None,
                            win32con.OPEN_EXISTING,
                            win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                            None
                        )
                    except pywintypes.error as e:
                        logging.error(f"Failed to open volume even with SeBackupPrivilege: {str(e)}")
                        return None
                else:
                    logging.error("Failed to enable SeBackupPrivilege")
                    return None
            else:
                logging.error(f"Unexpected error opening volume: {str(e)}")
                return None

    def enable_backup_privilege(self):
        try:
            h_token = win32security.OpenProcessToken(win32process.GetCurrentProcess(), win32security.TOKEN_ADJUST_PRIVILEGES | win32security.TOKEN_QUERY)
            luid = win32security.LookupPrivilegeValue(None, "SeBackupPrivilege")
            new_privileges = [(luid, win32security.SE_PRIVILEGE_ENABLED)]
            win32security.AdjustTokenPrivileges(h_token, False, new_privileges)
            return True
        except Exception as e:
            logging.error(f"Failed to enable SeBackupPrivilege: {str(e)}")
            return False

    def parse_mft_record(self, mft_record, record_number):
        if mft_record[0:4] != b'FILE':
            return

        flags = struct.unpack_from("<H", mft_record, 0x16)[0]
        if flags & 0x01 == 0:  # File is not in use (deleted)
            try:
                first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
                current_offset = first_attr_offset

                file_name = None
                file_size = 0
                mtime = None

                while current_offset < len(mft_record) - 4:
                    attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
                    if attr_type == 0x10:  # $STANDARD_INFORMATION
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_size = struct.unpack_from("<Q", mft_record, current_offset + 0x30)[0]
                        mtime = self.filetime_to_datetime(struct.unpack_from("<Q", mft_record, current_offset + 0x18)[0])
                    elif attr_type == 0x30:  # $FILE_NAME
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_name_length = struct.unpack_from("<B", mft_record, current_offset + 0x40)[0]
                        file_name = mft_record[current_offset + 0x42:current_offset + 0x42 + file_name_length * 2].decode('utf-16le')

                    current_offset += attr_length
                    if attr_length == 0:
                        break

                if file_name:
                    self.add_deleted_file(file_name, file_size, mtime, record_number)
            except Exception as e:
                logging.error(f"Failed to parse MFT record {record_number}: {str(e)}")

    def filetime_to_datetime(self, filetime):
        return datetime(1601, 1, 1) + timedelta(microseconds=filetime / 10)

    def add_deleted_file(self, name, size, mtime, record_number):
        self.deleted_files.append({
            'Name': name,
            'Size': size,
            'Date Modified': mtime,
            'MFT Record': record_number
        })
        self.master.after(0, self.update_file_tree)

    def update_file_tree(self):
        for i in self.file_tree.get_children():
            self.file_tree.delete(i)
        for file_info in self.deleted_files:
            self.file_tree.insert('', 'end', values=(file_info['Name'], file_info['Size'], file_info['Date Modified'], file_info['MFT Record']))

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
                messagebox.showinfo("Recovery Complete", "File recovery process is finished.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.recovery_queue.put("ERROR: Failed to open volume for recovery. Please run as administrator.")
                return

            for file_info in self.deleted_files:
                file_name = file_info['Name']
                mft_record = file_info['MFT Record']

                # Here you would implement actual file recovery logic.
                # For now, we'll just log the operation.
                logging.info(f"Recovering file {file_name} with MFT record {mft_record}...")
                self.recovery_queue.put(f"Recovering {file_name}...")

            win32file.CloseHandle(volume_handle)
            self.recovery_queue.put("RECOVERY_COMPLETE")
        except Exception as e:
            error_msg = f"Error recovering files: {str(e)}"
            logging.error(error_msg)
            self.recovery_queue.put(f"ERROR: {error_msg}")

if __name__ == "__main__":
    root = tk.Tk()
    app = NTFSFileRecoveryTool(root)
    root.mainloop()

In [1]:
import os
import struct
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import win32file
import win32con
import winioctlcon
import logging
from datetime import datetime, timedelta
import threading
import queue
import ctypes
import sys
import re
import hashlib
import csv
import pywintypes
import win32security
import win32process

logging.basicConfig(filename='ntfs_recovery.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def is_admin():
    try:
        return ctypes.windll.shell32.IsUserAnAdmin()
    except:
        return False

def request_admin():
    if not is_admin():
        ctypes.windll.shell32.ShellExecuteW(None, "runas", sys.executable, " ".join(sys.argv), None, 1)
        return True
    return False

class NTFSFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Advanced NTFS File Recovery Tool")
        self.master.geometry("1200x800")

        self.drive_letter = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select NTFS Drive (e.g., C:):").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=5, textvariable=self.drive_letter).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=2, row=0, sticky=tk.W, pady=5)
        ttk.Button(frame, text="Request Admin Rights", command=self.request_admin_rights).grid(column=3, row=0, sticky=tk.W, pady=5)

        self.file_tree = ttk.Treeview(frame, columns=('Name', 'Size', 'Date Modified', 'MFT Record'), show='headings')
        self.file_tree.heading('Name', text='Name')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.heading('MFT Record', text='MFT Record')
        self.file_tree.grid(column=0, row=1, columnspan=4, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=4, row=1, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=2, columnspan=4, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=3, columnspan=4, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=4, columnspan=4, sticky=(tk.W, tk.E), pady=5)

    def request_admin_rights(self):
        if request_admin():
            messagebox.showinfo("Admin Rights", "The application will restart with admin rights.")
            self.master.quit()
        else:
            messagebox.showinfo("Admin Rights", "The application is already running with admin rights.")

    def start_scan_thread(self):
        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())
        self.drive = self.drive_letter.get().upper()
        
        if not self.drive or len(self.drive) != 2 or self.drive[1] != ':':
            messagebox.showerror("Error", "Invalid drive letter")
            return

        scan_thread = threading.Thread(target=self.scan_ntfs)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_ntfs(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.scan_queue.put("ERROR: Failed to open volume. Please check your permissions.")
                return

            boot_sector = win32file.ReadFile(volume_handle, 512)[1]
            bytes_per_sector = struct.unpack_from("<H", boot_sector, 0x0B)[0]
            sectors_per_cluster = struct.unpack_from("<B", boot_sector, 0x0D)[0]
            mft_cluster = struct.unpack_from("<Q", boot_sector, 0x30)[0]
            mft_record_size = 1024  # Typically 1024 bytes for NTFS

            mft_offset = mft_cluster * sectors_per_cluster * bytes_per_sector

            win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
            
            for i in range(100000):  # Limit to first 100,000 MFT entries for this example
                try:
                    mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]
                    self.parse_mft_record(mft_record, i)
                except pywintypes.error as e:
                    if e.winerror == 23:  # ERROR_CRC
                        logging.warning(f"CRC error reading MFT record {i}, skipping...")
                        continue
                    else:
                        raise
                
                if i % 1000 == 0:
                    self.scan_queue.put(f"Scanned {i} MFT entries...")

            win32file.CloseHandle(volume_handle)
            self.scan_queue.put("SCAN_COMPLETE")
        except Exception as e:
            error_msg = f"Error scanning NTFS: {str(e)}"
            logging.error(error_msg)
            self.scan_queue.put(f"ERROR: {error_msg}")

    def open_volume(self):
        try:
            return win32file.CreateFile(
                f"\\\\.\\{self.drive}",
                win32con.GENERIC_READ,
                win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                None,
                win32con.OPEN_EXISTING,
                win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                None
            )
        except pywintypes.error as e:
            if e.winerror == 5:  # Access denied
                logging.warning("Access denied, attempting to enable SeBackupPrivilege...")
                if self.enable_backup_privilege():
                    try:
                        return win32file.CreateFile(
                            f"\\\\.\\{self.drive}",
                            win32con.GENERIC_READ,
                            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                            None,
                            win32con.OPEN_EXISTING,
                            win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                            None
                        )
                    except pywintypes.error as e:
                        logging.error(f"Failed to open volume even with SeBackupPrivilege: {str(e)}")
                        return None
                else:
                    logging.error("Failed to enable SeBackupPrivilege")
                    return None
            else:
                logging.error(f"Unexpected error opening volume: {str(e)}")
                return None

    def enable_backup_privilege(self):
        try:
            h_token = win32security.OpenProcessToken(win32process.GetCurrentProcess(), win32security.TOKEN_ADJUST_PRIVILEGES | win32security.TOKEN_QUERY)
            luid = win32security.LookupPrivilegeValue(None, "SeBackupPrivilege")
            new_privileges = [(luid, win32security.SE_PRIVILEGE_ENABLED)]
            win32security.AdjustTokenPrivileges(h_token, False, new_privileges)
            return True
        except Exception as e:
            logging.error(f"Failed to enable SeBackupPrivilege: {str(e)}")
            return False

    def parse_mft_record(self, mft_record, record_number):
        if mft_record[0:4] != b'FILE':
            return

        flags = struct.unpack_from("<H", mft_record, 0x16)[0]
        if flags & 0x01 == 0:  # File is not in use (deleted)
            try:
                first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
                current_offset = first_attr_offset

                file_name = None
                file_size = 0
                mtime = None

                while current_offset < len(mft_record) - 4:
                    attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
                    if attr_type == 0x10:  # $STANDARD_INFORMATION
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_size = struct.unpack_from("<Q", mft_record, current_offset + 0x30)[0]
                        mtime = self.filetime_to_datetime(struct.unpack_from("<Q", mft_record, current_offset + 0x18)[0])
                    elif attr_type == 0x30:  # $FILE_NAME
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_name_length = struct.unpack_from("<B", mft_record, current_offset + 0x40)[0]
                        file_name = mft_record[current_offset + 0x42:current_offset + 0x42 + file_name_length * 2].decode('utf-16le')

                    current_offset += attr_length
                    if attr_length == 0:
                        break

                if file_name:
                    self.add_deleted_file(file_name, file_size, mtime, record_number)
            except Exception as e:
                logging.error(f"Failed to parse MFT record {record_number}: {str(e)}")

    def filetime_to_datetime(self, filetime):
        return datetime(1601, 1, 1) + timedelta(microseconds=filetime / 10)

    def add_deleted_file(self, name, size, mtime, record_number):
        self.deleted_files.append({
            'Name': name,
            'Size': size,
            'Date Modified': mtime,
            'MFT Record': record_number
        })
        self.master.after(0, self.update_file_tree)

    def update_file_tree(self):
        for i in self.file_tree.get_children():
            self.file_tree.delete(i)
        for file_info in self.deleted_files:
            self.file_tree.insert('', 'end', values=(file_info['Name'], file_info['Size'], file_info['Date Modified'], file_info['MFT Record']))

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
                messagebox.showinfo("Recovery Complete", "File recovery process is finished.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.recovery_queue.put("ERROR: Failed to open volume for recovery. Please check your permissions.")
                return

            for file_info in self.deleted_files:
                file_name = file_info['Name']
                mft_record = file_info['MFT Record']

                # Here you would implement actual file recovery logic.
                # For now, we'll just log the operation.
                logging.info(f"Recovering file {file_name} with MFT record {mft_record}...")
                self.recovery_queue.put(f"Recovering {file_name}...")

            win32file.CloseHandle(volume_handle)
            self.recovery_queue.put("RECOVERY_COMPLETE")
        except Exception as e:
            error_msg = f"Error recovering files: {str(e)}"
            logging.error(error_msg)
            self.recovery_queue.put(f"ERROR: {error_msg}")

if __name__ == "__main__":
    root = tk.Tk()
    app = NTFSFileRecoveryTool(root)
    root.mainloop()

In [ ]:
import os
import struct
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import win32file
import win32con
import winioctlcon
import logging
from datetime import datetime, timedelta
import threading
import queue
import ctypes
import sys
import re
import hashlib
import csv
import pywintypes
import win32security
import win32process

logging.basicConfig(filename='ntfs_recovery.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def is_admin():
    try:
        return ctypes.windll.shell32.IsUserAnAdmin()
    except:
        return False

def request_admin():
    if not is_admin():
        ctypes.windll.shell32.ShellExecuteW(None, "runas", sys.executable, " ".join(sys.argv), None, 1)
        return True
    return False

class NTFSFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Advanced NTFS File Recovery Tool")
        self.master.geometry("1200x800")

        self.drive_letter = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select NTFS Drive (e.g., C:):").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=5, textvariable=self.drive_letter).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=2, row=0, sticky=tk.W, pady=5)
        ttk.Button(frame, text="Request Admin Rights", command=self.request_admin_rights).grid(column=3, row=0, sticky=tk.W, pady=5)

        self.file_tree = ttk.Treeview(frame, columns=('Name', 'Size', 'Date Modified', 'MFT Record'), show='headings')
        self.file_tree.heading('Name', text='Name')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.heading('MFT Record', text='MFT Record')
        self.file_tree.grid(column=0, row=1, columnspan=4, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=4, row=1, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=2, columnspan=4, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=3, columnspan=4, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=4, columnspan=4, sticky=(tk.W, tk.E), pady=5)

    def request_admin_rights(self):
        if request_admin():
            messagebox.showinfo("Admin Rights", "The application will restart with admin rights.")
            self.master.quit()
        else:
            messagebox.showinfo("Admin Rights", "The application is already running with admin rights.")

    def start_scan_thread(self):
        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())
        self.drive = self.drive_letter.get().upper()
        
        if not self.drive or len(self.drive) != 2 or self.drive[1] != ':':
            messagebox.showerror("Error", "Invalid drive letter")
            return

        scan_thread = threading.Thread(target=self.scan_ntfs)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_ntfs(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.scan_queue.put("ERROR: Failed to open volume. Please check your permissions.")
                return

            boot_sector = win32file.ReadFile(volume_handle, 512)[1]
            bytes_per_sector = struct.unpack_from("<H", boot_sector, 0x0B)[0]
            sectors_per_cluster = struct.unpack_from("<B", boot_sector, 0x0D)[0]
            mft_cluster = struct.unpack_from("<Q", boot_sector, 0x30)[0]
            mft_record_size = 1024  # Typically 1024 bytes for NTFS

            mft_offset = mft_cluster * sectors_per_cluster * bytes_per_sector

            win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
            
            for i in range(100000):  # Limit to first 100,000 MFT entries for this example
                try:
                    mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]
                    self.parse_mft_record(mft_record, i)
                except pywintypes.error as e:
                    if e.winerror == 23:  # ERROR_CRC
                        logging.warning(f"CRC error reading MFT record {i}, skipping...")
                        continue
                    else:
                        raise
                
                if i % 1000 == 0:
                    self.scan_queue.put(f"Scanned {i} MFT entries...")

            win32file.CloseHandle(volume_handle)
            self.scan_queue.put("SCAN_COMPLETE")
        except Exception as e:
            error_msg = f"Error scanning NTFS: {str(e)}"
            logging.error(error_msg)
            self.scan_queue.put(f"ERROR: {error_msg}")

    def open_volume(self):
        try:
            return win32file.CreateFile(
                f"\\\\.\\{self.drive}",
                win32con.GENERIC_READ,
                win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                None,
                win32con.OPEN_EXISTING,
                win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                None
            )
        except pywintypes.error as e:
            if e.winerror == 5:  # Access denied
                logging.warning("Access denied, attempting to enable SeBackupPrivilege...")
                if self.enable_backup_privilege():
                    try:
                        return win32file.CreateFile(
                            f"\\\\.\\{self.drive}",
                            win32con.GENERIC_READ,
                            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                            None,
                            win32con.OPEN_EXISTING,
                            win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                            None
                        )
                    except pywintypes.error as e:
                        logging.error(f"Failed to open volume even with SeBackupPrivilege: {str(e)}")
                        return None
                else:
                    logging.error("Failed to enable SeBackupPrivilege")
                    return None
            else:
                logging.error(f"Unexpected error opening volume: {str(e)}")
                return None

    def enable_backup_privilege(self):
        try:
            h_token = win32security.OpenProcessToken(win32process.GetCurrentProcess(), win32security.TOKEN_ADJUST_PRIVILEGES | win32security.TOKEN_QUERY)
            luid = win32security.LookupPrivilegeValue(None, "SeBackupPrivilege")
            new_privileges = [(luid, win32security.SE_PRIVILEGE_ENABLED)]
            win32security.AdjustTokenPrivileges(h_token, False, new_privileges)
            return True
        except Exception as e:
            logging.error(f"Failed to enable SeBackupPrivilege: {str(e)}")
            return False

    def parse_mft_record(self, mft_record, record_number):
        if mft_record[0:4] != b'FILE':
            return

        flags = struct.unpack_from("<H", mft_record, 0x16)[0]
        if flags & 0x01 == 0:  # File is not in use (deleted)
            try:
                first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
                current_offset = first_attr_offset

                file_name = None
                file_size = 0
                mtime = None

                while current_offset < len(mft_record) - 4:
                    attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
                    if attr_type == 0x10:  # $STANDARD_INFORMATION
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_size = struct.unpack_from("<Q", mft_record, current_offset + 0x30)[0]
                        mtime = self.filetime_to_datetime(struct.unpack_from("<Q", mft_record, current_offset + 0x18)[0])
                    elif attr_type == 0x30:  # $FILE_NAME
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_name_length = struct.unpack_from("<B", mft_record, current_offset + 0x40)[0]
                        file_name = mft_record[current_offset + 0x42:current_offset + 0x42 + file_name_length * 2].decode('utf-16le')

                    current_offset += attr_length
                    if attr_length == 0:
                        break

                if file_name:
                    self.add_deleted_file(file_name, file_size, mtime, record_number)
            except Exception as e:
                logging.error(f"Failed to parse MFT record {record_number}: {str(e)}")

    def filetime_to_datetime(self, filetime):
        return datetime(1601, 1, 1) + timedelta(microseconds=filetime / 10)

    def add_deleted_file(self, name, size, mtime, record_number):
        self.deleted_files.append({
            'Name': name,
            'Size': size,
            'Date Modified': mtime,
            'MFT Record': record_number
        })
        self.master.after(0, self.update_file_tree)

    def update_file_tree(self):
        for i in self.file_tree.get_children():
            self.file_tree.delete(i)
        for file_info in self.deleted_files:
            self.file_tree.insert('', 'end', values=(file_info['Name'], file_info['Size'], file_info['Date Modified'], file_info['MFT Record']))

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
                messagebox.showinfo("Recovery Complete", "File recovery process is finished.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.recovery_queue.put("ERROR: Failed to open volume for recovery. Please check your permissions.")
                return

            for file_info in self.deleted_files:
                file_name = file_info['Name']
                mft_record = file_info['MFT Record']

                # Here you would implement actual file recovery logic.
                # For now, we'll just log the operation.
                logging.info(f"Recovering file {file_name} with MFT record {mft_record}...")
                self.recovery_queue.put(f"Recovering {file_name}...")

            win32file.CloseHandle(volume_handle)
            self.recovery_queue.put("RECOVERY_COMPLETE")
        except Exception as e:
            error_msg = f"Error recovering files: {str(e)}"
            logging.error(error_msg)
            self.recovery_queue.put(f"ERROR: {error_msg}")

if __name__ == "__main__":
    root = tk.Tk()
    app = NTFSFileRecoveryTool(root)
    root.mainloop()
    
    
    
    #Working for admin

IndentationError: expected an indented block after function definition on line 115 (2877875980.py, line 116)

In [2]:
import os
import struct
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import win32file
import win32con
import winioctlcon
import logging
from datetime import datetime, timedelta
import threading
import queue
import ctypes
import sys
import re
import hashlib
import csv
import pywintypes
import win32security
import win32process

logging.basicConfig(filename='ntfs_recovery.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def is_admin():
    try:
        return ctypes.windll.shell32.IsUserAnAdmin()
    except:
        return False

def request_admin():
    if not is_admin():
        ctypes.windll.shell32.ShellExecuteW(None, "runas", sys.executable, " ".join(sys.argv), None, 1)
        return True
    return False

class NTFSFileRecoveryTool:
    def __init__(self, master):
        self.master = master
        self.master.title("Advanced NTFS File Recovery Tool")
        self.master.geometry("1200x800")

        self.drive_letter = tk.StringVar()
        self.deleted_files = []
        self.scan_queue = queue.Queue()
        self.recovery_queue = queue.Queue()

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.master, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.master.columnconfigure(0, weight=1)
        self.master.rowconfigure(0, weight=1)

        ttk.Label(frame, text="Select NTFS Drive (e.g., C:):").grid(column=0, row=0, sticky=tk.W, pady=5)
        ttk.Entry(frame, width=5, textvariable=self.drive_letter).grid(column=1, row=0, sticky=(tk.W, tk.E), pady=5)
        ttk.Button(frame, text="Scan for Deleted Files", command=self.start_scan_thread).grid(column=2, row=0, sticky=tk.W, pady=5)
        ttk.Button(frame, text="Request Admin Rights", command=self.request_admin_rights).grid(column=3, row=0, sticky=tk.W, pady=5)

        self.file_tree = ttk.Treeview(frame, columns=('Name', 'Size', 'Date Modified', 'MFT Record'), show='headings')
        self.file_tree.heading('Name', text='Name')
        self.file_tree.heading('Size', text='Size')
        self.file_tree.heading('Date Modified', text='Date Modified')
        self.file_tree.heading('MFT Record', text='MFT Record')
        self.file_tree.grid(column=0, row=1, columnspan=4, sticky=(tk.W, tk.E, tk.N, tk.S))
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.file_tree.yview)
        scrollbar.grid(column=4, row=1, sticky=(tk.N, tk.S))
        self.file_tree.configure(yscrollcommand=scrollbar.set)

        ttk.Button(frame, text="Recover Selected Files", command=self.start_recovery_thread).grid(column=0, row=2, columnspan=4, pady=10)

        self.progress = ttk.Progressbar(frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(column=0, row=3, columnspan=4, sticky=(tk.W, tk.E), pady=5)

        self.status_label = ttk.Label(frame, text="Ready")
        self.status_label.grid(column=0, row=4, columnspan=4, sticky=(tk.W, tk.E), pady=5)

    def request_admin_rights(self):
        if request_admin():
            messagebox.showinfo("Admin Rights", "The application will restart with admin rights.")
            self.master.quit()
        else:
            messagebox.showinfo("Admin Rights", "The application is already running with admin rights.")

    def start_scan_thread(self):
        self.deleted_files = []
        self.master.after(0, self.file_tree.delete, *self.file_tree.get_children())
        self.drive = self.drive_letter.get().upper()
        
        if not self.drive or len(self.drive) != 2 or self.drive[1] != ':':
            messagebox.showerror("Error", "Invalid drive letter")
            return

        scan_thread = threading.Thread(target=self.scan_ntfs)
        scan_thread.start()
        self.master.after(100, self.check_scan_queue)

    def check_scan_queue(self):
        try:
            message = self.scan_queue.get_nowait()
            if message == "SCAN_COMPLETE":
                self.status_label.config(text="Scan complete")
                messagebox.showinfo("Scan Complete", f"Found {len(self.deleted_files)} deleted files.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_scan_queue)
        except queue.Empty:
            self.master.after(100, self.check_scan_queue)

    def scan_ntfs(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.scan_queue.put("ERROR: Failed to open volume. Please check your permissions.")
                return

            boot_sector = win32file.ReadFile(volume_handle, 512)[1]
            bytes_per_sector = struct.unpack_from("<H", boot_sector, 0x0B)[0]
            sectors_per_cluster = struct.unpack_from("<B", boot_sector, 0x0D)[0]
            mft_cluster = struct.unpack_from("<Q", boot_sector, 0x30)[0]
            mft_record_size = 1024  # Typically 1024 bytes for NTFS

            mft_offset = mft_cluster * sectors_per_cluster * bytes_per_sector

            logging.info(f"MFT offset: {mft_offset}")
            self.scan_queue.put(f"MFT offset: {mft_offset}")

            win32file.SetFilePointer(volume_handle, mft_offset, win32file.FILE_BEGIN)
            
            total_records = 0
            deleted_records = 0

            for i in range(100000):  # Limit to first 100,000 MFT entries for this example
                try:
                    mft_record = win32file.ReadFile(volume_handle, mft_record_size)[1]
                    total_records += 1
                    is_deleted = self.parse_mft_record(mft_record, i)
                    if is_deleted:
                        deleted_records += 1
                except pywintypes.error as e:
                    if e.winerror == 23:  # ERROR_CRC
                        logging.warning(f"CRC error reading MFT record {i}, skipping...")
                        continue
                    else:
                        raise
                
                if i % 1000 == 0:
                    self.scan_queue.put(f"Scanned {i} MFT entries...")

            win32file.CloseHandle(volume_handle)
            logging.info(f"Total records scanned: {total_records}")
            logging.info(f"Deleted records found: {deleted_records}")
            self.scan_queue.put(f"Total records scanned: {total_records}")
            self.scan_queue.put(f"Deleted records found: {deleted_records}")
            self.scan_queue.put("SCAN_COMPLETE")
        except Exception as e:
            error_msg = f"Error scanning NTFS: {str(e)}"
            logging.error(error_msg)
            self.scan_queue.put(f"ERROR: {error_msg}")

    def open_volume(self):
        try:
            return win32file.CreateFile(
                f"\\\\.\\{self.drive}",
                win32con.GENERIC_READ,
                win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                None,
                win32con.OPEN_EXISTING,
                win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                None
            )
        except pywintypes.error as e:
            if e.winerror == 5:  # Access denied
                logging.warning("Access denied, attempting to enable SeBackupPrivilege...")
                if self.enable_backup_privilege():
                    try:
                        return win32file.CreateFile(
                            f"\\\\.\\{self.drive}",
                            win32con.GENERIC_READ,
                            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
                            None,
                            win32con.OPEN_EXISTING,
                            win32con.FILE_ATTRIBUTE_NORMAL | win32con.FILE_FLAG_BACKUP_SEMANTICS,
                            None
                        )
                    except pywintypes.error as e:
                        logging.error(f"Failed to open volume even with SeBackupPrivilege: {str(e)}")
                        return None
                else:
                    logging.error("Failed to enable SeBackupPrivilege")
                    return None
            else:
                logging.error(f"Unexpected error opening volume: {str(e)}")
                return None

    def enable_backup_privilege(self):
        try:
            h_token = win32security.OpenProcessToken(win32process.GetCurrentProcess(), win32security.TOKEN_ADJUST_PRIVILEGES | win32security.TOKEN_QUERY)
            luid = win32security.LookupPrivilegeValue(None, "SeBackupPrivilege")
            new_privileges = [(luid, win32security.SE_PRIVILEGE_ENABLED)]
            win32security.AdjustTokenPrivileges(h_token, False, new_privileges)
            return True
        except Exception as e:
            logging.error(f"Failed to enable SeBackupPrivilege: {str(e)}")
            return False

    def parse_mft_record(self, mft_record, record_number):
        if mft_record[0:4] != b'FILE':
            return False

        flags = struct.unpack_from("<H", mft_record, 0x16)[0]
        is_deleted = flags & 0x01 == 0  # File is not in use (deleted)

        if is_deleted:
            try:
                first_attr_offset = struct.unpack_from("<H", mft_record, 0x14)[0]
                current_offset = first_attr_offset

                file_name = None
                file_size = 0
                mtime = None

                while current_offset < len(mft_record) - 4:
                    attr_type = struct.unpack_from("<I", mft_record, current_offset)[0]
                    if attr_type == 0x10:  # $STANDARD_INFORMATION
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_size = struct.unpack_from("<Q", mft_record, current_offset + 0x30)[0]
                        mtime = self.filetime_to_datetime(struct.unpack_from("<Q", mft_record, current_offset + 0x18)[0])
                    elif attr_type == 0x30:  # $FILE_NAME
                        attr_length = struct.unpack_from("<I", mft_record, current_offset + 0x04)[0]
                        file_name_length = struct.unpack_from("<B", mft_record, current_offset + 0x40)[0]
                        file_name = mft_record[current_offset + 0x42:current_offset + 0x42 + file_name_length * 2].decode('utf-16le')

                    current_offset += attr_length
                    if attr_length == 0:
                        break

                if file_name:
                    self.add_deleted_file(file_name, file_size, mtime, record_number)
                    logging.info(f"Found deleted file: {file_name}")
                else:
                    logging.warning(f"Deleted record {record_number} found, but no file name extracted")
            except Exception as e:
                logging.error(f"Failed to parse MFT record {record_number}: {str(e)}")

        return is_deleted

    def filetime_to_datetime(self, filetime):
        return datetime(1601, 1, 1) + timedelta(microseconds=filetime / 10)

    def add_deleted_file(self, name, size, mtime, record_number):
        self.deleted_files.append({
            'Name': name,
            'Size': size,
            'Date Modified': mtime,
            'MFT Record': record_number
        })
        self.master.after(0, self.update_file_tree)

    def update_file_tree(self):
        for i in self.file_tree.get_children():
            self.file_tree.delete(i)
        for file_info in self.deleted_files:
            self.file_tree.insert('', 'end', values=(file_info['Name'], file_info['Size'], file_info['Date Modified'], file_info['MFT Record']))

    def start_recovery_thread(self):
        recovery_thread = threading.Thread(target=self.recover_files)
        recovery_thread.start()
        self.master.after(100, self.check_recovery_queue)

    def check_recovery_queue(self):
        try:
            message = self.recovery_queue.get_nowait()
            if message == "RECOVERY_COMPLETE":
                self.status_label.config(text="Recovery complete")
                messagebox.showinfo("Recovery Complete", "File recovery process is finished.")
            elif message.startswith("ERROR:"):
                self.status_label.config(text=message)
                messagebox.showerror("Error", message)
            else:
                self.status_label.config(text=message)
                self.master.after(100, self.check_recovery_queue)
        except queue.Empty:
            self.master.after(100, self.check_recovery_queue)

    def recover_files(self):
        try:
            volume_handle = self.open_volume()
            if not volume_handle:
                self.recovery_queue.put("ERROR: Failed to open volume for recovery. Please check your permissions.")
                return

            for file_info in self.deleted_files:
                file_name = file_info['Name']
                mft_record = file_info['MFT Record']

                # Here you would implement actual file recovery logic.
                # For now, we'll just log the operation.
                logging.info(f"Recovering file {file_name} with MFT record {mft_record}...")
                self.recovery_queue.put(f"Recovering {file_name}...")

            win32file.CloseHandle(volume_handle)
            self.recovery_queue.put("RECOVERY_COMPLETE")
        except Exception as e:
            error_msg = f"Error recovering files: {str(e)}"
            logging.error(error_msg)
            self.recovery_queue.put(f"ERROR: {error_msg}")

if __name__ == "__main__":
    root = tk.Tk()
    app = NTFSFileRecoveryTool(root)
    root.mainloop()